# Solutions Engineering Workload Modeling

## Project Overview

This Jupyter notebook models Solutions Engineering workload based on revenue targets, conversion rates, staffing levels, and activity assumptions. It enables scenario modeling to understand capacity requirements and utilization across different business conditions.

### Key Capabilities
- **Revenue-Driven Pipeline Modeling**: Calculate required activities from quarterly revenue goals
- **Interactive Parameter Controls**: Adjust assumptions and see real-time impact
- **Capacity Analysis**: Understand SE team utilization and identify constraints
- **Account Management Modeling**: Factor in existing customer meeting requirements
- **Strategic Activity Planning**: Allocate time for non-pipeline activities
- **Professional Visualizations**: Executive-ready charts and dashboards

### Business Applications
- Staffing planning and capacity forecasting
- Revenue target feasibility analysis
- Resource allocation optimization
- Scenario planning for different business conditions

## Introduction

### Workload Modeling Approach

This model uses a **revenue-driven approach** to calculate SE workload requirements:

1. **Start with Revenue Goals**: Quarterly revenue targets drive required new logo counts
2. **Work Backwards Through Funnel**: Calculate required demos, evaluations, and meetings
3. **Factor in Activity Types**: Different evaluation types require different SE time investments
4. **Add Account Management**: Existing customer meetings and onboarding requirements
5. **Include Strategic Activities**: Time for customer zero, content creation, and enablement
6. **Calculate Utilization**: Compare total workload to available SE capacity

### Key Assumptions
- **40-hour work week** standard capacity per SE
- **Conversion rates** remain consistent across evaluation types
- **Account meeting frequencies** based on customer segment strategy
- **Strategic activities** distributed between ICs and Directors based on role allocation

## Library Imports and Configuration

In [6]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Interactive widgets for parameter controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure matplotlib for inline display
%matplotlib inline

# Set seaborn style for professional appearance
sns.set_style("whitegrid")
sns.set_palette("husl")

# Configure pandas display options for better table formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 2)

# Configure matplotlib for better chart appearance
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11

print("✅ All libraries imported successfully")
print("✅ Configuration applied")
print("📊 Ready to begin workload modeling")

✅ All libraries imported successfully
✅ Configuration applied
📊 Ready to begin workload modeling


## Table of Contents

### 📋 Core Sections
1. [**Helper Functions Framework**](#helper-functions) - Core calculation functions
2. [**Parameter Management**](#parameter-management) - Input parameter definitions and validation
3. [**Revenue & Pipeline Calculations**](#revenue-pipeline) - Core funnel math and workload calculations
4. [**Account Management Logic**](#account-management) - Meeting frequency and time allocation
5. [**Strategic Activities Logic**](#strategic-activities) - Strategic time allocation and distribution

### 🎛️ Interactive Interface
6. [**Interactive Parameter Controls**](#interactive-controls) - Widget-based parameter input
7. [**Real-time Calculations**](#real-time-calculations) - Live calculation updates

### 📊 Visualization & Results
8. [**Core Visualizations**](#core-visualizations) - Essential charts for utilization and capacity
9. [**Executive Dashboard**](#executive-dashboard) - Professional dashboard with multiple chart types
10. [**Results Summary & Export**](#results-summary) - Executive summary and export capabilities

### 🔧 Testing & Validation
11. [**Testing Framework**](#testing) - Comprehensive validation and testing
12. [**Usage Examples**](#examples) - Sample scenarios and use cases

---

# 1. Helper Functions Framework {#helper-functions}

*Core calculation functions that will be reused throughout the notebook*

In [7]:
# Core helper functions for SE workload modeling calculations

def calculate_required_new_logos(quarterly_revenue_goal, average_selling_price):
    """
    Calculate the number of new logos needed to meet quarterly revenue goals.
    
    Parameters:
    - quarterly_revenue_goal (float): Target revenue for the quarter
    - average_selling_price (float): Average deal size
    
    Returns:
    - float: Number of new logos required
    """
    if average_selling_price <= 0:
        raise ValueError("Average selling price must be greater than 0")
    
    return quarterly_revenue_goal / average_selling_price


def calculate_weighted_win_rate(eval_mix_percentages, win_rates):
    """
    Calculate weighted average win rate across all evaluation types.
    
    Parameters:
    - eval_mix_percentages (dict): Percentages for each evaluation type
    - win_rates (dict): Win rates for each evaluation type
    
    Returns:
    - float: Weighted average win rate (0-1)
    """
    total_percentage = sum(eval_mix_percentages.values())
    if abs(total_percentage - 100) > 0.01:  # Allow small floating point errors
        raise ValueError(f"Evaluation mix percentages must sum to 100%, got {total_percentage}%")
    
    weighted_sum = 0
    for eval_type in eval_mix_percentages:
        if eval_type not in win_rates:
            raise ValueError(f"Win rate not provided for evaluation type: {eval_type}")
        weighted_sum += (eval_mix_percentages[eval_type] / 100) * (win_rates[eval_type] / 100)
    
    return weighted_sum


def calculate_required_activities(new_logos_needed, conversion_rates, eval_mix, win_rates):
    """
    Work backwards through conversion funnel to calculate required weekly activities.
    
    Parameters:
    - new_logos_needed (float): Required new logos per quarter
    - conversion_rates (dict): Conversion rates for each funnel stage
    - eval_mix (dict): Percentage mix of evaluation types
    - win_rates (dict): Win rates by evaluation type
    
    Returns:
    - dict: Required weekly activities by type
    """
    # Calculate weighted win rate
    weighted_win_rate = calculate_weighted_win_rate(eval_mix, win_rates)
    
    # Work backwards through funnel (quarterly to weekly)
    weeks_per_quarter = 13
    
    # Required evaluations per quarter
    required_evaluations_quarterly = new_logos_needed / weighted_win_rate
    
    # Required demos per quarter (from tech eval conversion rate)
    required_demos_quarterly = required_evaluations_quarterly / (conversion_rates['tech_eval_conversion_rate'] / 100)
    
    # Required opportunities per quarter (from demo conversion rate)
    required_opportunities_quarterly = required_demos_quarterly / (conversion_rates['demo_conversion_rate'] / 100)
    
    # Required initial meetings per quarter (from opportunity creation rate)
    required_meetings_quarterly = required_opportunities_quarterly / (conversion_rates['opportunity_creation_rate'] / 100)
    
    # Convert to weekly and break down evaluations by type
    weekly_activities = {
        'initial_meetings': required_meetings_quarterly / weeks_per_quarter,
        'demos': required_demos_quarterly / weeks_per_quarter,
        'total_evaluations': required_evaluations_quarterly / weeks_per_quarter
    }
    
    # Break down evaluations by type
    for eval_type, percentage in eval_mix.items():
        eval_key = f"{eval_type.replace('_percentage', '')}_evaluations"
        weekly_activities[eval_key] = weekly_activities['total_evaluations'] * (percentage / 100)
    
    return weekly_activities


def calculate_evaluation_workload(weekly_evaluations_by_type, time_per_type):
    """
    Calculate SE time requirements for different evaluation types.
    
    Parameters:
    - weekly_evaluations_by_type (dict): Weekly evaluation counts by type
    - time_per_type (dict): Time requirements for each evaluation type
    
    Returns:
    - dict: Weekly hours needed by evaluation type
    """
    evaluation_hours = {}
    
    # Self-guided: ongoing weekly support hours
    if 'self_guided' in weekly_evaluations_by_type:
        evaluation_hours['self_guided'] = (weekly_evaluations_by_type['self_guided'] * 
                                         time_per_type['self_guided_support_hours_per_week'])
    
    # SE-led: ongoing weekly support hours  
    if 'se_led' in weekly_evaluations_by_type:
        evaluation_hours['se_led'] = (weekly_evaluations_by_type['se_led'] * 
                                    time_per_type['se_led_eval_hours_per_week'])
    
    # Rapid POV: total hours distributed over evaluation period (assume 2 weeks)
    if 'rapid_pov' in weekly_evaluations_by_type:
        rapid_pov_weeks = 2  # Standard duration for rapid POV
        evaluation_hours['rapid_pov'] = (weekly_evaluations_by_type['rapid_pov'] * 
                                       time_per_type['rapid_pov_total_hours'] / rapid_pov_weeks)
    
    # No evaluation type requires no SE time
    evaluation_hours['no_eval'] = 0
    
    return evaluation_hours


def calculate_demo_workload(weekly_demos, demo_time_params):
    """
    Calculate total demo workload including prep, delivery, and follow-up.
    
    Parameters:
    - weekly_demos (float): Number of demos per week
    - demo_time_params (dict): Time parameters for demo activities
    
    Returns:
    - float: Total weekly hours for demo activities
    """
    total_demo_time_per_demo = (demo_time_params['demo_prep_time'] + 
                               demo_time_params['demo_delivery_time'] + 
                               demo_time_params['demo_followup_time'])
    
    return weekly_demos * total_demo_time_per_demo


def calculate_meeting_workload(weekly_meetings, meeting_time):
    """
    Calculate workload for initial meetings including prep time.
    
    Parameters:
    - weekly_meetings (float): Number of initial meetings per week
    - meeting_time (float): Time per meeting including prep
    
    Returns:
    - float: Total weekly hours for initial meetings
    """
    return weekly_meetings * meeting_time


def calculate_account_meetings(account_counts, meeting_frequencies):
    """
    Calculate total meetings per month for existing accounts.
    
    Parameters:
    - account_counts (dict): Number of accounts by segment
    - meeting_frequencies (dict): Meeting frequency by segment (meetings/month)
    
    Returns:
    - dict: Monthly meetings by account segment
    """
    monthly_meetings = {}
    
    for segment, count in account_counts.items():
        if segment in meeting_frequencies:
            if segment == 'retain':
                # Retain accounts: frequency is "every X months", so meetings/month = 1/X
                meetings_per_month = count / meeting_frequencies[segment]
            else:
                # Other segments: frequency is meetings per month
                meetings_per_month = count * meeting_frequencies[segment]
            
            monthly_meetings[segment] = meetings_per_month
    
    return monthly_meetings


def calculate_new_logo_onboarding_meetings(new_logos_per_quarter, onboarding_params):
    """
    Calculate new customer onboarding meeting requirements.
    
    Parameters:
    - new_logos_per_quarter (float): New customers per quarter
    - onboarding_params (dict): Onboarding meeting parameters
    
    Returns:
    - dict: Monthly onboarding meeting requirements
    """
    monthly_onboarding = {}
    
    # Monthly meetings: 50% of new logos need 6 months of meetings
    monthly_customers = (new_logos_per_quarter * 
                        (onboarding_params['monthly_onboarding_percentage'] / 100))
    # 6 months of meetings spread across year = 6/12 = 0.5 factor
    monthly_onboarding['monthly_meetings'] = monthly_customers * 0.5
    
    # Quarterly meetings: 50% of new logos need 4 meetings per year
    quarterly_customers = (new_logos_per_quarter * 
                          (onboarding_params['quarterly_onboarding_percentage'] / 100))
    # 4 meetings per year = 4/12 meetings per month
    monthly_onboarding['quarterly_meetings'] = quarterly_customers * (4/12)
    
    return monthly_onboarding


def calculate_account_mgmt_hours(monthly_meetings, meeting_time_params):
    """
    Convert monthly meetings to weekly SE hours.
    
    Parameters:
    - monthly_meetings (dict): Monthly meeting counts by type
    - meeting_time_params (dict): Time per meeting including follow-up
    
    Returns:
    - float: Weekly hours for account management
    """
    total_monthly_meetings = sum(monthly_meetings.values())
    
    time_per_meeting = (meeting_time_params['meeting_duration_hours'] + 
                       meeting_time_params['meeting_followup_hours'])
    
    monthly_hours = total_monthly_meetings * time_per_meeting
    
    # Convert to weekly hours (assume 4.33 weeks per month)
    weekly_hours = monthly_hours / 4.33
    
    return weekly_hours


def calculate_strategic_workload(strategic_params, staffing_config):
    """
    Calculate strategic activity hours per role.
    
    Parameters:
    - strategic_params (dict): Strategic activity time parameters
    - staffing_config (dict): Team structure configuration
    
    Returns:
    - dict: Strategic hours by role type
    """
    # Calculate total strategic hours per week
    total_strategic_hours = 0
    
    # Handle min/max ranges by taking midpoint
    customer_zero_hours = (strategic_params['customer_zero_hours_min'] + 
                          strategic_params['customer_zero_hours_max']) / 2
    
    developer_advocacy_hours = (strategic_params['developer_advocacy_hours_min'] + 
                               strategic_params['developer_advocacy_hours_max']) / 2
    
    total_strategic_hours = (customer_zero_hours + 
                           strategic_params['content_creation_hours'] +
                           strategic_params['sales_enablement_hours'] +
                           developer_advocacy_hours +
                           strategic_params['asset_development_hours_avg'])
    
    return distribute_strategic_work(total_strategic_hours, 
                                   staffing_config['ic_strategic_percentage'],
                                   staffing_config['num_ic_ses'],
                                   staffing_config['num_directors'],
                                   staffing_config['director_ic_percentage'])


def distribute_strategic_work(total_hours, ic_percentage, num_ics, num_directors, director_ic_percentage):
    """
    Distribute strategic work between ICs and Directors.
    
    Parameters:
    - total_hours (float): Total strategic hours per week
    - ic_percentage (float): Percentage allocated to ICs
    - num_ics (int): Number of IC SEs
    - num_directors (int): Number of directors
    - director_ic_percentage (float): Percentage of director time on IC activities
    
    Returns:
    - dict: Strategic hours by role
    """
    ic_strategic_hours = total_hours * (ic_percentage / 100)
    director_strategic_hours = total_hours * ((100 - ic_percentage) / 100)
    
    # Calculate effective director capacity for strategic work
    # Directors spend (100 - director_ic_percentage)% on strategic activities
    director_strategic_capacity_percentage = 100 - director_ic_percentage
    
    return {
        'ic_strategic_hours_per_se': ic_strategic_hours / max(num_ics, 1),
        'director_strategic_hours_per_director': director_strategic_hours / max(num_directors, 1),
        'total_ic_strategic_hours': ic_strategic_hours,
        'total_director_strategic_hours': director_strategic_hours,
        'director_strategic_capacity_percentage': director_strategic_capacity_percentage
    }


def validate_percentages_sum_to_100(percentage_dict, tolerance=0.01):
    """
    Validate that percentages sum to 100%.
    
    Parameters:
    - percentage_dict (dict): Dictionary of percentage values
    - tolerance (float): Allowed deviation from 100%
    
    Returns:
    - bool: True if valid, False otherwise
    """
    total = sum(percentage_dict.values())
    return abs(total - 100) <= tolerance


def calculate_weekly_capacity(num_ses, hours_per_week=40):
    """
    Calculate total weekly capacity for SE team.
    
    Parameters:
    - num_ses (float): Number of SEs (can be fractional for part-time)
    - hours_per_week (float): Standard hours per week per SE
    
    Returns:
    - float: Total weekly capacity in hours
    """
    return num_ses * hours_per_week


def format_hours_for_display(hours):
    """
    Format hours for user-friendly display.
    
    Parameters:
    - hours (float): Hours to format
    
    Returns:
    - str: Formatted string
    """
    if hours < 0.1:
        return "< 0.1 hrs"
    elif hours < 1:
        return f"{hours:.1f} hrs"
    else:
        return f"{hours:.1f} hrs"


# Test helper functions with sample data
def test_helper_functions():
    """Test all helper functions with sample inputs to verify correctness."""
    print("🧪 Testing Helper Functions")
    print("=" * 40)
    
    test_results = []
    
    try:
        # Test revenue calculation
        result = calculate_required_new_logos(2000000, 75000)
        expected = 26.67
        assert abs(result - expected) < 0.1, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_required_new_logos: Working correctly")
        
        # Test win rate calculation
        eval_mix = {'self_guided_percentage': 65, 'se_led_percentage': 35}
        win_rates = {'self_guided_percentage': 35, 'se_led_percentage': 45}
        result = calculate_weighted_win_rate(eval_mix, win_rates)
        expected = 0.385  # (0.65 * 0.35) + (0.35 * 0.45)
        assert abs(result - expected) < 0.01, f"Expected ~{expected}, got {result}"
        test_results.append("✅ calculate_weighted_win_rate: Working correctly")
        
        # Test capacity calculation
        result = calculate_weekly_capacity(2.5, 40)
        expected = 100
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ calculate_weekly_capacity: Working correctly")
        
        # Test percentage validation
        valid_percentages = {'a': 30, 'b': 70}
        invalid_percentages = {'a': 30, 'b': 80}
        assert validate_percentages_sum_to_100(valid_percentages) == True
        assert validate_percentages_sum_to_100(invalid_percentages) == False
        test_results.append("✅ validate_percentages_sum_to_100: Working correctly")
        
        # Test hours formatting
        result = format_hours_for_display(12.567)
        expected = "12.6 hrs"
        assert result == expected, f"Expected {expected}, got {result}"
        test_results.append("✅ format_hours_for_display: Working correctly")
        
    except Exception as e:
        test_results.append(f"❌ Helper function test failed: {str(e)}")
    
    # Display results
    for result in test_results:
        print(result)
    
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print(f"\n📊 Test Summary: {success_count}/{total_count} tests passed")
    
    if success_count == total_count:
        print("🎉 All helper function tests passed!")
        return True
    else:
        print("⚠️  Some helper function tests failed.")
        return False

# Run tests
test_helper_functions()

print("\n✅ Helper Functions Framework implemented successfully")
print("📝 Ready for Step 3: Parameter Management")

🧪 Testing Helper Functions
✅ calculate_required_new_logos: Working correctly
✅ calculate_weighted_win_rate: Working correctly
✅ calculate_weekly_capacity: Working correctly
✅ validate_percentages_sum_to_100: Working correctly
✅ format_hours_for_display: Working correctly

📊 Test Summary: 5/5 tests passed
🎉 All helper function tests passed!

✅ Helper Functions Framework implemented successfully
📝 Ready for Step 3: Parameter Management


---

# 2. Parameter Management {#parameter-management}

*Input parameter definitions, default values, and validation rules*

In [ ]:
# Comprehensive parameter management system with default values and validation

# ===================================
# PARAMETER DICTIONARIES WITH DEFAULTS
# ===================================

# Revenue and Business Target Parameters
revenue_params = {
    'quarterly_revenue_goal': 2000000,      # $2M quarterly revenue target
    'average_selling_price': 75000          # $75K average deal size
}

# Sales Conversion Funnel Parameters
conversion_params = {
    'opportunity_creation_rate': 50,        # 50% of meetings create opportunities
    'demo_conversion_rate': 99,             # 99% of opportunities get demos
    'tech_eval_conversion_rate': 99         # 99% of demos get technical evaluation
}

# Evaluation Mix Parameters (must sum to 100%)
eval_mix_params = {
    'self_guided_percentage': 64.0,         # Self-guided evaluations
    'se_led_percentage': 35.0,              # SE-led evaluations  
    'rapid_pov_percentage': 1.0,            # Rapid POV evaluations
    'no_eval_percentage': 0.0               # No evaluation deals
}

# Win Rate Parameters by Evaluation Type
win_rate_params = {
    'self_guided_win_rate': 35,             # 35% win rate for self-guided
    'se_led_win_rate': 45,                  # 45% win rate for SE-led
    'rapid_pov_win_rate': 60,               # 60% win rate for rapid POV
    'no_eval_win_rate': 10                  # 10% win rate for no evaluation
}

# Activity Time Requirements (hours)
activity_time_params = {
    'initial_meeting_time': 0.75,           # Initial meeting + prep time
    'demo_prep_time': 1.0,                  # Demo preparation time
    'demo_delivery_time': 1.0,              # Demo delivery time  
    'demo_followup_time': 0.75,             # Demo follow-up time
    'self_guided_support_hours_per_week': 2.0,     # Weekly support for self-guided
    'se_led_eval_hours_per_week': 4.0,             # Weekly hours for SE-led evals
    'rapid_pov_total_hours': 30             # Total hours for rapid POV
}

# Non-Pipeline Activity Hours (fixed weekly activities per SE)
fixed_activity_params = {
    'slack_email_hours': 5.0,               # Slack/email communication
    'troubleshooting_hours': 3.75,          # Technical troubleshooting
    'internal_coordination_hours': 7.75,    # Internal meetings/coordination
    'crm_admin_hours': 2.5                  # CRM and administrative tasks
}

# Team Structure and Staffing Parameters
staffing_params = {
    'num_ic_ses': 1,                        # Number of IC SEs
    'num_directors': 1,                     # Number of SE Directors
    'director_ic_percentage': 50            # % of director time on IC activities
}

# Strategic Activity Parameters (hours per week per FTE SE)
strategic_params = {
    'customer_zero_hours_min': 2.0,         # Customer zero minimum hours
    'customer_zero_hours_max': 4.0,         # Customer zero maximum hours
    'content_creation_hours': 2.0,          # Content creation hours
    'sales_enablement_hours': 1.0,          # Sales enablement hours
    'developer_advocacy_hours_min': 1.0,    # Developer advocacy minimum
    'developer_advocacy_hours_max': 2.0,    # Developer advocacy maximum
    'asset_development_hours_avg': 1.5,     # Asset development average hours
    'ic_strategic_percentage': 70           # % of strategic work for ICs vs Directors
}

# Account Portfolio Parameters
account_params = {
    'strategic_accounts': 3,                # Number of strategic accounts
    'protect_accounts': 7,                  # Number of protect accounts
    'growth_accounts': 24,                  # Number of growth accounts
    'retain_accounts': 59                   # Number of retain accounts
}

# Meeting Frequency Parameters
meeting_frequency_params = {
    'strategic_meetings_per_month': 1.0,    # Strategic account meetings/month
    'protect_meetings_per_month': 1.0,      # Protect account meetings/month
    'growth_meetings_per_month': 1.0,       # Growth account meetings/month
    'retain_meetings_per_months': 9         # Retain accounts: every X months
}

# New Customer Onboarding Parameters
onboarding_params = {
    'new_logos_per_quarter': 7,             # Expected new logos per quarter
    'monthly_onboarding_percentage': 50,    # % needing monthly meetings (6 months)
    'quarterly_onboarding_percentage': 50   # % needing quarterly meetings (4/year)
}

# Meeting Time Investment Parameters
meeting_time_params = {
    'meeting_duration_hours': 0.5,          # Base meeting duration
    'meeting_followup_hours': 0.625         # Follow-up time per meeting
}

# ===================================
# PARAMETER VALIDATION FUNCTIONS
# ===================================

def validate_eval_mix_percentages():
    """Validate that evaluation mix percentages sum to 100%."""
    total = sum(eval_mix_params.values())
    tolerance = 0.01  # Allow small floating point errors
    
    if abs(total - 100) > tolerance:
        return False, f"Evaluation mix percentages sum to {total:.1f}%, must equal 100%"
    return True, "Evaluation mix percentages valid"

def validate_onboarding_percentages():
    """Validate that onboarding percentages sum to ≤100%."""
    total = (onboarding_params['monthly_onboarding_percentage'] + 
             onboarding_params['quarterly_onboarding_percentage'])
    
    if total > 100:
        return False, f"Onboarding percentages sum to {total}%, must be ≤100%"
    return True, "Onboarding percentages valid"

def validate_positive_values():
    """Validate that all time and count parameters are positive."""
    
    # Check all parameter dictionaries for positive values
    param_groups = [
        ('revenue_params', revenue_params),
        ('conversion_params', conversion_params),
        ('activity_time_params', activity_time_params),
        ('fixed_activity_params', fixed_activity_params),
        ('staffing_params', staffing_params),
        ('strategic_params', strategic_params),
        ('account_params', account_params),
        ('meeting_frequency_params', meeting_frequency_params),
        ('meeting_time_params', meeting_time_params)
    ]
    
    errors = []
    
    for group_name, params in param_groups:
        for param_name, value in params.items():
            if value < 0:
                errors.append(f"{group_name}.{param_name}: {value} (must be ≥ 0)")
    
    if errors:
        return False, f"Negative values found: {', '.join(errors)}"
    return True, "All values are positive"

def validate_percentage_bounds():
    """Validate that all percentage parameters are between 0-100%."""
    
    # Percentage parameters to check
    percentage_params = [
        ('conversion_params.opportunity_creation_rate', conversion_params['opportunity_creation_rate']),
        ('conversion_params.demo_conversion_rate', conversion_params['demo_conversion_rate']),
        ('conversion_params.tech_eval_conversion_rate', conversion_params['tech_eval_conversion_rate']),
        ('eval_mix_params.self_guided_percentage', eval_mix_params['self_guided_percentage']),
        ('eval_mix_params.se_led_percentage', eval_mix_params['se_led_percentage']),
        ('eval_mix_params.rapid_pov_percentage', eval_mix_params['rapid_pov_percentage']),
        ('eval_mix_params.no_eval_percentage', eval_mix_params['no_eval_percentage']),
        ('win_rate_params.self_guided_win_rate', win_rate_params['self_guided_win_rate']),
        ('win_rate_params.se_led_win_rate', win_rate_params['se_led_win_rate']),
        ('win_rate_params.rapid_pov_win_rate', win_rate_params['rapid_pov_win_rate']),
        ('win_rate_params.no_eval_win_rate', win_rate_params['no_eval_win_rate']),
        ('staffing_params.director_ic_percentage', staffing_params['director_ic_percentage']),
        ('strategic_params.ic_strategic_percentage', strategic_params['ic_strategic_percentage']),
        ('onboarding_params.monthly_onboarding_percentage', onboarding_params['monthly_onboarding_percentage']),
        ('onboarding_params.quarterly_onboarding_percentage', onboarding_params['quarterly_onboarding_percentage'])
    ]
    
    errors = []
    
    for param_name, value in percentage_params:
        if not (0 <= value <= 100):
            errors.append(f"{param_name}: {value}% (must be 0-100%)")
    
    if errors:
        return False, f"Invalid percentage values: {', '.join(errors)}"
    return True, "All percentage values are valid"

def validate_logical_constraints():
    """Validate logical business constraints."""
    warnings = []
    errors = []
    
    # Check for unrealistic conversion rates
    if conversion_params['opportunity_creation_rate'] > 80:
        warnings.append(f"High opportunity creation rate: {conversion_params['opportunity_creation_rate']}%")
    
    # Check for unrealistic win rates
    avg_win_rate = sum(win_rate_params.values()) / len(win_rate_params)
    if avg_win_rate > 60:
        warnings.append(f"High average win rate: {avg_win_rate:.1f}%")
    
    # Check for unrealistic team structure
    total_directors = staffing_params['num_directors']
    total_ics = staffing_params['num_ic_ses']
    if total_directors > total_ics and total_ics > 0:
        warnings.append(f"More directors ({total_directors}) than ICs ({total_ics})")
    
    # Check for account load reasonableness
    total_accounts = sum(account_params.values())
    total_se_capacity = total_ics + total_directors
    if total_accounts > 0 and total_se_capacity > 0:
        accounts_per_se = total_accounts / total_se_capacity
        if accounts_per_se > 50:
            warnings.append(f"High account load: {accounts_per_se:.1f} accounts per SE")
    
    return errors, warnings

# ===================================
# CENTRAL PARAMETER UPDATE FUNCTION
# ===================================

def update_all_parameters():
    """Apply comprehensive validation and return validation results."""
    print("🔍 Validating Parameters")
    print("=" * 30)
    
    validation_results = []
    warnings = []
    
    # Run all validation checks
    validations = [
        validate_eval_mix_percentages(),
        validate_onboarding_percentages(),
        validate_positive_values(),
        validate_percentage_bounds()
    ]
    
    # Check validation results
    all_valid = True
    for is_valid, message in validations:
        if is_valid:
            validation_results.append(f"✅ {message}")
        else:
            validation_results.append(f"❌ {message}")
            all_valid = False
    
    # Check logical constraints
    errors, constraint_warnings = validate_logical_constraints()
    
    for error in errors:
        validation_results.append(f"❌ {error}")
        all_valid = False
    
    warnings.extend(constraint_warnings)
    
    # Display results
    for result in validation_results:
        print(result)
    
    if warnings:
        print("\n⚠️  Warnings:")
        for warning in warnings:
            print(f"   • {warning}")
    
    print(f"\n📊 Validation Summary:")
    if all_valid:
        print("✅ All parameters valid and ready for calculations")
        if warnings:
            print(f"⚠️  {len(warnings)} warnings (calculations will proceed)")
    else:
        print("❌ Parameter validation failed - please correct errors")
    
    return all_valid, warnings

# ===================================
# PARAMETER SUMMARY DISPLAY
# ===================================

def display_parameter_summary():
    """Display a comprehensive summary of all current parameters."""
    print("📋 Parameter Summary")
    print("=" * 50)
    
    print("\n💰 Revenue & Business Targets:")
    print(f"   • Quarterly Revenue Goal: ${revenue_params['quarterly_revenue_goal']:,}")
    print(f"   • Average Selling Price: ${revenue_params['average_selling_price']:,}")
    
    print("\n🔄 Sales Conversion Funnel:")
    for param, value in conversion_params.items():
        print(f"   • {param.replace('_', ' ').title()}: {value}%")
    
    print("\n📊 Evaluation Mix:")
    for param, value in eval_mix_params.items():
        print(f"   • {param.replace('_', ' ').title()}: {value}%")
    
    print("\n🎯 Win Rates by Evaluation Type:")
    for param, value in win_rate_params.items():
        print(f"   • {param.replace('_', ' ').title()}: {value}%")
    
    print("\n⏰ Activity Time Requirements:")
    for param, value in activity_time_params.items():
        print(f"   • {param.replace('_', ' ').title()}: {value} hrs")
    
    print("\n👥 Team Structure:")
    print(f"   • IC SEs: {staffing_params['num_ic_ses']}")
    print(f"   • Directors: {staffing_params['num_directors']}")
    print(f"   • Director IC Time: {staffing_params['director_ic_percentage']}%")
    
    print("\n🏢 Account Portfolio:")
    total_accounts = sum(account_params.values())
    for param, value in account_params.items():
        percentage = (value / total_accounts * 100) if total_accounts > 0 else 0
        print(f"   • {param.replace('_', ' ').title()}: {value} ({percentage:.1f}%)")
    
    print(f"\n📈 Strategic Activities (IC Focus: {strategic_params['ic_strategic_percentage']}%):")
    print(f"   • Customer Zero: {strategic_params['customer_zero_hours_min']}-{strategic_params['customer_zero_hours_max']} hrs/week")
    print(f"   • Content Creation: {strategic_params['content_creation_hours']} hrs/week")
    print(f"   • Sales Enablement: {strategic_params['sales_enablement_hours']} hrs/week")
    print(f"   • Developer Advocacy: {strategic_params['developer_advocacy_hours_min']}-{strategic_params['developer_advocacy_hours_max']} hrs/week")
    print(f"   • Asset Development: {strategic_params['asset_development_hours_avg']} hrs/week (avg)")

# ===================================
# PARAMETER TESTING
# ===================================

def test_parameter_management():
    """Test parameter management system with various scenarios."""
    print("🧪 Testing Parameter Management System")
    print("=" * 45)
    
    test_results = []
    
    try:
        # Test 1: Default parameters should be valid
        is_valid, warnings = update_all_parameters()
        if is_valid:
            test_results.append("✅ Default parameters: Valid")
        else:
            test_results.append("❌ Default parameters: Invalid")
        
        # Test 2: Test evaluation mix validation
        original_mix = eval_mix_params.copy()
        eval_mix_params['self_guided_percentage'] = 50
        eval_mix_params['se_led_percentage'] = 60  # This should make total > 100%
        
        is_valid, message = validate_eval_mix_percentages()
        if not is_valid:
            test_results.append("✅ Evaluation mix validation: Correctly detects invalid sum")
        else:
            test_results.append("❌ Evaluation mix validation: Failed to detect invalid sum")
        
        # Restore original values
        eval_mix_params.update(original_mix)
        
        # Test 3: Test percentage bounds validation
        original_conversion = conversion_params['opportunity_creation_rate']
        conversion_params['opportunity_creation_rate'] = 150  # Invalid percentage
        
        is_valid, message = validate_percentage_bounds()
        if not is_valid:
            test_results.append("✅ Percentage bounds validation: Correctly detects out-of-bounds")
        else:
            test_results.append("❌ Percentage bounds validation: Failed to detect out-of-bounds")
        
        # Restore original value
        conversion_params['opportunity_creation_rate'] = original_conversion
        
        # Test 4: Test positive values validation
        original_revenue = revenue_params['quarterly_revenue_goal']
        revenue_params['quarterly_revenue_goal'] = -1000000  # Invalid negative
        
        is_valid, message = validate_positive_values()
        if not is_valid:
            test_results.append("✅ Positive values validation: Correctly detects negative values")
        else:
            test_results.append("❌ Positive values validation: Failed to detect negative values")
        
        # Restore original value
        revenue_params['quarterly_revenue_goal'] = original_revenue
        
    except Exception as e:
        test_results.append(f"❌ Parameter management test failed: {str(e)}")
    
    # Display test results
    print("\n📊 Test Results:")
    for result in test_results:
        print(result)
    
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print(f"\n📈 Test Summary: {success_count}/{total_count} tests passed")
    
    if success_count == total_count:
        print("🎉 All parameter management tests passed!")
        return True
    else:
        print("⚠️  Some parameter management tests failed.")
        return False

# ===================================
# INITIALIZATION
# ===================================

# Run initial validation and display
print("🔧 Parameter Management System Initialized")
print("=" * 50)

# Test the parameter management system
test_parameter_management()

print("\n" + "=" * 50)
# Validate current parameters
is_valid, warnings = update_all_parameters()

if is_valid:
    print("\n📋 Current Parameter Configuration:")
    display_parameter_summary()
    print("\n✅ Parameter Management System ready for use")
    print("📝 Ready for Step 4: Revenue & Pipeline Calculations")
else:
    print("\n❌ Please correct parameter validation errors before proceeding")
    print("📝 Fix errors and re-run parameter validation")

---

# 3. Revenue & Pipeline Calculations {#revenue-pipeline}

*Core funnel math and workload calculations driven by revenue targets*

In [9]:
# Revenue & Pipeline Calculation Engine
# Core business logic for revenue-driven workload modeling

# ===================================
# PIPELINE CALCULATION ENGINE
# ===================================

def run_pipeline_calculations():
    """
    Execute complete pipeline calculation flow from revenue targets to SE workload.
    
    Returns:
    - dict: Comprehensive pipeline analysis results
    """
    print("💰 Running Revenue & Pipeline Calculations")
    print("=" * 50)
    
    # Step 1: Revenue-to-Activity Calculation
    print("\n📊 Step 1: Revenue Target Analysis")
    print("-" * 35)
    
    # Calculate required new logos
    new_logos_needed = calculate_required_new_logos(
        revenue_params['quarterly_revenue_goal'],
        revenue_params['average_selling_price']
    )
    
    print(f"💰 Revenue Goal: ${revenue_params['quarterly_revenue_goal']:,}/quarter")
    print(f"💵 Average Deal Size: ${revenue_params['average_selling_price']:,}")
    print(f"🎯 Required New Logos: {new_logos_needed:.1f}/quarter ({new_logos_needed/13:.1f}/week)")
    
    # Step 2: Pipeline Funnel Calculations
    print("\n🔄 Step 2: Pipeline Funnel Analysis")
    print("-" * 38)
    
    # Calculate weighted win rate
    weighted_win_rate = calculate_weighted_win_rate(eval_mix_params, win_rate_params)
    
    print(f"📈 Weighted Win Rate: {weighted_win_rate:.1%}")
    print("\n🎲 Win Rate Breakdown:")
    for eval_type, percentage in eval_mix_params.items():
        win_rate_key = eval_type.replace('_percentage', '_win_rate')
        if win_rate_key in win_rate_params and percentage > 0:
            print(f"   • {eval_type.replace('_', ' ').title()}: {percentage:.1f}% of mix @ {win_rate_params[win_rate_key]}% win rate")
    
    # Calculate required activities using helper function
    weekly_activities = calculate_required_activities(
        new_logos_needed, 
        conversion_params, 
        eval_mix_params, 
        win_rate_params
    )
    
    print("\n📅 Required Weekly Activities:")
    print(f"   • Initial Meetings: {weekly_activities['initial_meetings']:.1f}/week")
    print(f"   • Demos: {weekly_activities['demos']:.1f}/week")
    print(f"   • Total Evaluations: {weekly_activities['total_evaluations']:.1f}/week")
    
    # Step 3: Evaluation Workload Breakdown
    print("\n🔬 Step 3: Evaluation Workload Analysis")
    print("-" * 42)
    
    # Break down evaluations by type
    evaluation_breakdown = {}
    for eval_type, percentage in eval_mix_params.items():
        if percentage > 0:
            eval_key = eval_type.replace('_percentage', '')
            count = weekly_activities['total_evaluations'] * (percentage / 100)
            evaluation_breakdown[eval_key] = count
            print(f"   • {eval_type.replace('_', ' ').title()}: {count:.1f}/week")
    
    # Calculate time requirements for each evaluation type
    evaluation_hours = calculate_evaluation_workload(evaluation_breakdown, activity_time_params)
    
    print("\n⏰ Evaluation Time Requirements (per week):")
    total_eval_hours = 0
    for eval_type, hours in evaluation_hours.items():
        if hours > 0:
            print(f"   • {eval_type.replace('_', ' ').title()}: {hours:.1f} hrs")
            total_eval_hours += hours
    print(f"   • Total Evaluation Hours: {total_eval_hours:.1f} hrs/week")
    
    # Step 4: Demo and Meeting Workload
    print("\n🎤 Step 4: Demo & Meeting Workload Analysis")
    print("-" * 45)
    
    # Calculate demo workload
    demo_hours = calculate_demo_workload(weekly_activities['demos'], activity_time_params)
    print(f"🎯 Demo Workload: {demo_hours:.1f} hrs/week")
    print(f"   • Prep: {weekly_activities['demos'] * activity_time_params['demo_prep_time']:.1f} hrs")
    print(f"   • Delivery: {weekly_activities['demos'] * activity_time_params['demo_delivery_time']:.1f} hrs")
    print(f"   • Follow-up: {weekly_activities['demos'] * activity_time_params['demo_followup_time']:.1f} hrs")
    
    # Calculate meeting workload
    meeting_hours = calculate_meeting_workload(
        weekly_activities['initial_meetings'], 
        activity_time_params['initial_meeting_time']
    )
    print(f"🤝 Initial Meetings: {meeting_hours:.1f} hrs/week ({weekly_activities['initial_meetings']:.1f} meetings)")
    
    # Step 5: Pipeline Activity Summary
    print("\n📋 Step 5: Pipeline Activity Summary")
    print("-" * 37)
    
    total_pipeline_hours = total_eval_hours + demo_hours + meeting_hours
    
    pipeline_summary = {
        'revenue_goal': revenue_params['quarterly_revenue_goal'],
        'new_logos_needed': new_logos_needed,
        'weighted_win_rate': weighted_win_rate,
        'weekly_activities': weekly_activities,
        'evaluation_breakdown': evaluation_breakdown,
        'evaluation_hours': evaluation_hours,
        'demo_hours': demo_hours,
        'meeting_hours': meeting_hours,
        'total_pipeline_hours': total_pipeline_hours
    }
    
    print(f"💼 Total Pipeline Hours: {total_pipeline_hours:.1f} hrs/week")
    print(f"   • Meetings: {meeting_hours:.1f} hrs ({meeting_hours/total_pipeline_hours:.1%})")
    print(f"   • Demos: {demo_hours:.1f} hrs ({demo_hours/total_pipeline_hours:.1%})")
    print(f"   • Evaluations: {total_eval_hours:.1f} hrs ({total_eval_hours/total_pipeline_hours:.1%})")
    
    return pipeline_summary

def calculate_pipeline_efficiency_metrics(pipeline_results):
    """
    Calculate efficiency and performance metrics for the pipeline.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - dict: Efficiency metrics and insights
    """
    
    print("\n📊 Pipeline Efficiency Analysis")
    print("=" * 35)
    
    # Calculate key ratios
    weekly_activities = pipeline_results['weekly_activities']
    
    # Conversion efficiency through funnel
    conversion_efficiency = {
        'meetings_to_opportunities': conversion_params['opportunity_creation_rate'] / 100,
        'opportunities_to_demos': conversion_params['demo_conversion_rate'] / 100,
        'demos_to_evaluations': conversion_params['tech_eval_conversion_rate'] / 100,
        'evaluations_to_wins': pipeline_results['weighted_win_rate']
    }
    
    # Overall funnel efficiency (meetings to wins)
    overall_efficiency = 1
    for stage, rate in conversion_efficiency.items():
        overall_efficiency *= rate
    
    print(f"🎯 Overall Funnel Efficiency: {overall_efficiency:.1%}")
    print(f"   • Meetings → Opportunities: {conversion_efficiency['meetings_to_opportunities']:.1%}")
    print(f"   • Opportunities → Demos: {conversion_efficiency['opportunities_to_demos']:.1%}")
    print(f"   • Demos → Evaluations: {conversion_efficiency['demos_to_evaluations']:.1%}")
    print(f"   • Evaluations → Wins: {conversion_efficiency['evaluations_to_wins']:.1%}")
    
    # Time investment per new logo
    time_per_new_logo = pipeline_results['total_pipeline_hours'] / (pipeline_results['new_logos_needed'] / 13)
    
    print(f"\n⏱️  Time Investment Metrics:")
    print(f"   • Hours per New Logo: {time_per_new_logo:.1f} hrs")
    print(f"   • Hours per $1M Revenue: {pipeline_results['total_pipeline_hours'] / (revenue_params['quarterly_revenue_goal'] / 1000000 / 13):.1f} hrs/week")
    
    # Activity intensity metrics
    print(f"\n📈 Activity Intensity:")
    print(f"   • Meetings per Day: {weekly_activities['initial_meetings'] / 5:.1f}")
    print(f"   • Demos per Day: {weekly_activities['demos'] / 5:.1f}")
    print(f"   • Evaluations per Day: {weekly_activities['total_evaluations'] / 5:.1f}")
    
    efficiency_metrics = {
        'conversion_efficiency': conversion_efficiency,
        'overall_efficiency': overall_efficiency,
        'time_per_new_logo': time_per_new_logo,
        'hours_per_million_revenue': pipeline_results['total_pipeline_hours'] / (revenue_params['quarterly_revenue_goal'] / 1000000 / 13)
    }
    
    return efficiency_metrics

def create_pipeline_dataframe(pipeline_results):
    """
    Create a comprehensive DataFrame summarizing pipeline calculations.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - pd.DataFrame: Structured pipeline data
    """
    
    # Create pipeline summary table
    pipeline_data = []
    
    # Revenue and target data
    pipeline_data.append({
        'Category': 'Revenue Targets',
        'Metric': 'Quarterly Revenue Goal',
        'Weekly Value': revenue_params['quarterly_revenue_goal'] / 13,
        'Quarterly Value': revenue_params['quarterly_revenue_goal'],
        'Unit': '$',
        'Notes': f'${revenue_params["quarterly_revenue_goal"]:,} target'
    })
    
    pipeline_data.append({
        'Category': 'Revenue Targets', 
        'Metric': 'Required New Logos',
        'Weekly Value': pipeline_results['new_logos_needed'] / 13,
        'Quarterly Value': pipeline_results['new_logos_needed'],
        'Unit': 'logos',
        'Notes': f'${revenue_params["average_selling_price"]:,} avg deal size'
    })
    
    # Activity requirements
    for activity, count in pipeline_results['weekly_activities'].items():
        if activity != 'total_evaluations':  # Skip total since we have breakdown
            pipeline_data.append({
                'Category': 'Required Activities',
                'Metric': activity.replace('_', ' ').title(),
                'Weekly Value': count,
                'Quarterly Value': count * 13,
                'Unit': 'activities',
                'Notes': f'{count:.1f} per week required'
            })
    
    # Time requirements
    time_data = [
        ('Meeting Hours', pipeline_results['meeting_hours'], 'meetings'),
        ('Demo Hours', pipeline_results['demo_hours'], 'demos'),
        ('Total Evaluation Hours', sum(pipeline_results['evaluation_hours'].values()), 'evaluations')
    ]
    
    for metric, hours, source in time_data:
        pipeline_data.append({
            'Category': 'Time Requirements',
            'Metric': metric,
            'Weekly Value': hours,
            'Quarterly Value': hours * 13,
            'Unit': 'hours',
            'Notes': f'From {source}'
        })
    
    # Total pipeline workload
    pipeline_data.append({
        'Category': 'Total Workload',
        'Metric': 'Total Pipeline Hours',
        'Weekly Value': pipeline_results['total_pipeline_hours'],
        'Quarterly Value': pipeline_results['total_pipeline_hours'] * 13,
        'Unit': 'hours',
        'Notes': 'All pipeline activities combined'
    })
    
    df = pd.DataFrame(pipeline_data)
    
    print("\n📊 Pipeline Summary Table")
    print("=" * 30)
    
    # Display formatted table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    
    # Format numeric columns
    display_df = df.copy()
    display_df['Weekly Value'] = display_df['Weekly Value'].apply(lambda x: f'{x:.1f}')
    display_df['Quarterly Value'] = display_df['Quarterly Value'].apply(lambda x: f'{x:.1f}')
    
    print(display_df.to_string(index=False))
    
    return df

def validate_pipeline_calculations(pipeline_results):
    """
    Validate pipeline calculations for reasonableness and logical consistency.
    
    Parameters:
    - pipeline_results (dict): Results from run_pipeline_calculations()
    
    Returns:
    - tuple: (is_valid, validation_messages)
    """
    
    print("\n🔍 Pipeline Validation")
    print("=" * 25)
    
    validation_messages = []
    warnings = []
    is_valid = True
    
    # Check for reasonable activity levels
    weekly_activities = pipeline_results['weekly_activities']
    
    # Validate meeting frequency (should be realistic for SE capacity)
    if weekly_activities['initial_meetings'] > 50:
        warnings.append(f"High meeting load: {weekly_activities['initial_meetings']:.1f} meetings/week")
    
    if weekly_activities['demos'] > 25:
        warnings.append(f"High demo load: {weekly_activities['demos']:.1f} demos/week")
    
    # Check total hours against reasonable SE capacity
    total_hours = pipeline_results['total_pipeline_hours']
    if total_hours > 40:
        warnings.append(f"Pipeline hours exceed full-time capacity: {total_hours:.1f} hrs/week")
    
    # Validate win rate reasonableness
    if pipeline_results['weighted_win_rate'] > 0.7:
        warnings.append(f"Very high win rate: {pipeline_results['weighted_win_rate']:.1%}")
    elif pipeline_results['weighted_win_rate'] < 0.2:
        warnings.append(f"Very low win rate: {pipeline_results['weighted_win_rate']:.1%}")
    
    # Check for mathematical consistency
    expected_total_evals = sum(pipeline_results['evaluation_breakdown'].values())
    actual_total_evals = weekly_activities['total_evaluations']
    
    if abs(expected_total_evals - actual_total_evals) > 0.01:
        validation_messages.append(f"❌ Evaluation breakdown inconsistency: {expected_total_evals:.2f} vs {actual_total_evals:.2f}")
        is_valid = False
    else:
        validation_messages.append("✅ Evaluation breakdown mathematically consistent")
    
    # Validate conversion rates produce expected results
    meetings = weekly_activities['initial_meetings']
    opportunities = meetings * (conversion_params['opportunity_creation_rate'] / 100)
    expected_demos = opportunities * (conversion_params['demo_conversion_rate'] / 100)
    
    if abs(expected_demos - weekly_activities['demos']) > 0.01:
        validation_messages.append(f"❌ Demo calculation inconsistency")
        is_valid = False
    else:
        validation_messages.append("✅ Funnel calculations mathematically consistent")
    
    # Display results
    for message in validation_messages:
        print(message)
    
    if warnings:
        print("\n⚠️  Warnings:")
        for warning in warnings:
            print(f"   • {warning}")
    
    print(f"\n📊 Validation Summary:")
    if is_valid:
        print("✅ Pipeline calculations validated successfully")
        if warnings:
            print(f"⚠️  {len(warnings)} warnings noted")
    else:
        print("❌ Pipeline validation failed - check calculations")
    
    return is_valid, validation_messages + warnings

# ===================================
# SCENARIO ANALYSIS FUNCTIONS
# ===================================

def run_sensitivity_analysis():
    """Run sensitivity analysis on key pipeline parameters."""
    
    print("\n🎭 Pipeline Sensitivity Analysis")
    print("=" * 35)
    
    base_results = run_pipeline_calculations()
    base_hours = base_results['total_pipeline_hours']
    
    print(f"\n📊 Base Case: {base_hours:.1f} hrs/week")
    
    # Test sensitivity to key parameters
    sensitivities = []
    
    # Revenue goal sensitivity
    original_revenue = revenue_params['quarterly_revenue_goal']
    for multiplier in [0.5, 0.75, 1.25, 1.5]:
        revenue_params['quarterly_revenue_goal'] = original_revenue * multiplier
        test_results = run_pipeline_calculations()
        change = (test_results['total_pipeline_hours'] - base_hours) / base_hours
        sensitivities.append({
            'Parameter': 'Revenue Goal',
            'Change': f'{multiplier:.0%} of base',
            'Hours': test_results['total_pipeline_hours'],
            'Change %': f'{change:+.1%}'
        })
    revenue_params['quarterly_revenue_goal'] = original_revenue
    
    # Win rate sensitivity  
    original_win_rates = win_rate_params.copy()
    for multiplier in [0.8, 0.9, 1.1, 1.2]:
        for rate_key in win_rate_params:
            win_rate_params[rate_key] = min(100, original_win_rates[rate_key] * multiplier)
        test_results = run_pipeline_calculations()
        change = (test_results['total_pipeline_hours'] - base_hours) / base_hours
        sensitivities.append({
            'Parameter': 'Win Rates',
            'Change': f'{multiplier:.0%} of base',
            'Hours': test_results['total_pipeline_hours'],
            'Change %': f'{change:+.1%}'
        })
    win_rate_params.update(original_win_rates)
    
    # Display sensitivity results
    sensitivity_df = pd.DataFrame(sensitivities)
    print("\n📈 Sensitivity Results:")
    print(sensitivity_df.to_string(index=False))
    
    return sensitivity_df

# ===================================
# MAIN EXECUTION
# ===================================

# Run the complete pipeline calculation flow
pipeline_results = run_pipeline_calculations()

# Calculate efficiency metrics
efficiency_metrics = calculate_pipeline_efficiency_metrics(pipeline_results)

# Create summary DataFrame
pipeline_df = create_pipeline_dataframe(pipeline_results)

# Validate calculations
is_valid, validation_messages = validate_pipeline_calculations(pipeline_results)

# Store results for use in other sections
PIPELINE_RESULTS = {
    'pipeline_summary': pipeline_results,
    'efficiency_metrics': efficiency_metrics,
    'pipeline_df': pipeline_df,
    'validation_status': is_valid,
    'validation_messages': validation_messages
}

if is_valid:
    print("\n✅ Revenue & Pipeline Calculations completed successfully")
    print("📝 Ready for Step 5: Account Management Logic")
    
    # Run sensitivity analysis
    sensitivity_results = run_sensitivity_analysis()
    PIPELINE_RESULTS['sensitivity_analysis'] = sensitivity_results
    
else:
    print("\n❌ Pipeline calculations need review before proceeding")
    print("📝 Please address validation issues")

💰 Running Revenue & Pipeline Calculations

📊 Step 1: Revenue Target Analysis
-----------------------------------
💰 Revenue Goal: $2,000,000/quarter
💵 Average Deal Size: $75,000
🎯 Required New Logos: 26.7/quarter (2.1/week)

🔄 Step 2: Pipeline Funnel Analysis
--------------------------------------


ValueError: Evaluation mix percentages must sum to 100%, got 101.0%

---

# 4. Account Management Logic {#account-management}

*Meeting frequency calculations and time allocation for existing customers*

In [ ]:
# Account Management Logic Engine
# Meeting frequency calculations and time allocation for customer relationships

# ===================================
# ACCOUNT MANAGEMENT CALCULATION ENGINE
# ===================================

def run_account_management_calculations():
    """
    Execute complete account management calculation flow for existing and new customers.
    
    Returns:
    - dict: Comprehensive account management analysis results
    """
    print("👥 Running Account Management Calculations")
    print("=" * 50)
    
    # Step 1: Existing Account Meeting Analysis
    print("\n🏢 Step 1: Existing Account Portfolio Analysis")
    print("-" * 48)
    
    # Calculate total accounts and distribution
    total_accounts = sum(account_params.values())
    
    print(f"📊 Account Portfolio Summary:")
    for segment, count in account_params.items():
        percentage = (count / total_accounts * 100) if total_accounts > 0 else 0
        print(f"   • {segment.replace('_', ' ').title()}: {count} accounts ({percentage:.1f}%)")
    print(f"   • Total Portfolio: {total_accounts} accounts")
    
    # Calculate monthly meetings by segment using helper function
    monthly_meetings_by_segment = calculate_account_meetings(account_params, meeting_frequency_params)
    
    print(f"\n📅 Monthly Meeting Requirements by Segment:")
    total_monthly_meetings = 0
    for segment, meetings in monthly_meetings_by_segment.items():
        total_monthly_meetings += meetings
        frequency = meeting_frequency_params.get(f'{segment}_meetings_per_month', 
                                                meeting_frequency_params.get(f'{segment}_meetings_per_months', 'N/A'))
        print(f"   • {segment.replace('_', ' ').title()}: {meetings:.1f} meetings/month")
        if segment == 'retain':
            print(f"     (Every {frequency} months)")
        else:
            print(f"     ({frequency} meetings/month per account)")
    
    print(f"   • Total Existing Accounts: {total_monthly_meetings:.1f} meetings/month")
    
    # Step 2: New Customer Onboarding Analysis
    print("\n🆕 Step 2: New Customer Onboarding Analysis")
    print("-" * 45)
    
    # Use the expected new logos from pipeline calculations if available
    if 'PIPELINE_RESULTS' in globals() and 'pipeline_summary' in PIPELINE_RESULTS:
        new_logos = PIPELINE_RESULTS['pipeline_summary']['new_logos_needed']
        print(f"📈 Using pipeline-calculated new logos: {new_logos:.1f}/quarter")
    else:
        new_logos = onboarding_params['new_logos_per_quarter']
        print(f"📊 Using parameter-based new logos: {new_logos}/quarter")
    
    # Calculate onboarding meetings using helper function
    onboarding_meetings = calculate_new_logo_onboarding_meetings(new_logos, onboarding_params)
    
    print(f"\n🎯 New Customer Onboarding Breakdown:")
    total_onboarding_monthly = 0
    for meeting_type, count in onboarding_meetings.items():
        total_onboarding_monthly += count
        if 'monthly' in meeting_type:
            print(f"   • Monthly Meetings: {count:.1f}/month")
            print(f"     ({onboarding_params['monthly_onboarding_percentage']}% of new logos × 6 months)")
        else:
            print(f"   • Quarterly Meetings: {count:.1f}/month")
            print(f"     ({onboarding_params['quarterly_onboarding_percentage']}% of new logos × 4/year)")
    
    print(f"   • Total New Customer Meetings: {total_onboarding_monthly:.1f} meetings/month")
    
    # Step 3: Combined Meeting Load Analysis
    print("\n🔄 Step 3: Combined Meeting Load Analysis")
    print("-" * 42)
    
    # Combine all meetings
    all_monthly_meetings = monthly_meetings_by_segment.copy()
    all_monthly_meetings.update(onboarding_meetings)
    
    total_all_meetings = total_monthly_meetings + total_onboarding_monthly
    
    print(f"📋 Total Account Management Load:")
    print(f"   • Existing Accounts: {total_monthly_meetings:.1f} meetings/month ({total_monthly_meetings/total_all_meetings:.1%})")
    print(f"   • New Customer Onboarding: {total_onboarding_monthly:.1f} meetings/month ({total_onboarding_monthly/total_all_meetings:.1%})")
    print(f"   • Combined Total: {total_all_meetings:.1f} meetings/month")
    
    # Step 4: Time Allocation Calculations
    print("\n⏰ Step 4: Time Allocation Analysis")
    print("-" * 36)
    
    # Calculate weekly hours using helper function
    weekly_account_hours = calculate_account_mgmt_hours(all_monthly_meetings, meeting_time_params)
    
    # Break down time components
    meeting_duration = meeting_time_params['meeting_duration_hours']
    followup_time = meeting_time_params['meeting_followup_hours']
    total_time_per_meeting = meeting_duration + followup_time
    
    print(f"⏱️  Time Investment per Meeting:")
    print(f"   • Meeting Duration: {meeting_duration} hrs")
    print(f"   • Follow-up Time: {followup_time} hrs")
    print(f"   • Total per Meeting: {total_time_per_meeting} hrs")
    
    print(f"\n📊 Weekly Time Requirements:")
    print(f"   • Total Account Management: {weekly_account_hours:.1f} hrs/week")
    print(f"   • Average per Meeting: {total_time_per_meeting:.2f} hrs")
    print(f"   • Weekly Meetings: {total_all_meetings / 4.33:.1f} meetings")
    
    # Step 5: Account Management Summary
    account_summary = {
        'total_accounts': total_accounts,
        'account_distribution': account_params,
        'monthly_meetings_by_segment': monthly_meetings_by_segment,
        'new_logos_quarterly': new_logos,
        'onboarding_meetings': onboarding_meetings,
        'total_monthly_meetings': total_all_meetings,
        'weekly_account_hours': weekly_account_hours,
        'time_per_meeting': total_time_per_meeting,
        'weekly_meetings': total_all_meetings / 4.33
    }
    
    return account_summary

def calculate_account_load_distribution(account_results):
    """
    Calculate how account management load distributes across SE team.
    
    Parameters:
    - account_results (dict): Results from run_account_management_calculations()
    
    Returns:
    - dict: Account load distribution analysis
    """
    
    print("\n👨‍💼 Account Load Distribution Analysis")
    print("=" * 40)
    
    # Calculate effective SE capacity for account management
    total_ics = staffing_params['num_ic_ses']
    total_directors = staffing_params['num_directors']
    director_ic_time = staffing_params['director_ic_percentage'] / 100
    
    # Effective account management capacity
    effective_account_managers = total_ics + (total_directors * director_ic_time)
    
    print(f"👥 Team Structure:")
    print(f"   • IC SEs: {total_ics}")
    print(f"   • Directors: {total_directors}")
    print(f"   • Director IC Time: {staffing_params['director_ic_percentage']}%")
    print(f"   • Effective Account Managers: {effective_account_managers:.1f}")
    
    # Calculate load per SE
    if effective_account_managers > 0:
        accounts_per_se = account_results['total_accounts'] / effective_account_managers
        meetings_per_se_weekly = account_results['weekly_meetings'] / effective_account_managers
        hours_per_se_weekly = account_results['weekly_account_hours'] / effective_account_managers
        
        print(f"\n📊 Load per SE:")
        print(f"   • Accounts per SE: {accounts_per_se:.1f}")
        print(f"   • Meetings per SE per Week: {meetings_per_se_weekly:.1f}")
        print(f"   • Hours per SE per Week: {hours_per_se_weekly:.1f}")
        
        # Capacity warnings
        warnings = []
        if accounts_per_se > 50:
            warnings.append(f"Very high account load: {accounts_per_se:.1f} accounts per SE")
        elif accounts_per_se > 30:
            warnings.append(f"High account load: {accounts_per_se:.1f} accounts per SE")
        
        if hours_per_se_weekly > 20:
            warnings.append(f"Account management time exceeds 50% capacity: {hours_per_se_weekly:.1f} hrs/week")
        elif hours_per_se_weekly > 16:
            warnings.append(f"High account management time: {hours_per_se_weekly:.1f} hrs/week (40% capacity)")
        
        if warnings:
            print(f"\n⚠️  Load Distribution Warnings:")
            for warning in warnings:
                print(f"   • {warning}")
        else:
            print(f"\n✅ Account load distribution appears reasonable")
    
    else:
        accounts_per_se = 0
        meetings_per_se_weekly = 0
        hours_per_se_weekly = 0
        warnings = ["No SEs available for account management"]
    
    distribution_results = {
        'effective_account_managers': effective_account_managers,
        'accounts_per_se': accounts_per_se,
        'meetings_per_se_weekly': meetings_per_se_weekly,
        'hours_per_se_weekly': hours_per_se_weekly,
        'warnings': warnings
    }
    
    return distribution_results

def create_account_management_dataframe(account_results, distribution_results):
    """
    Create comprehensive DataFrame summarizing account management calculations.
    
    Parameters:
    - account_results (dict): Results from run_account_management_calculations()
    - distribution_results (dict): Results from calculate_account_load_distribution()
    
    Returns:
    - pd.DataFrame: Structured account management data
    """
    
    account_data = []
    
    # Account portfolio data
    for segment, count in account_results['account_distribution'].items():
        percentage = (count / account_results['total_accounts'] * 100) if account_results['total_accounts'] > 0 else 0
        account_data.append({
            'Category': 'Account Portfolio',
            'Metric': f'{segment.replace("_", " ").title()} Accounts',
            'Count': count,
            'Percentage': f'{percentage:.1f}%',
            'Monthly Meetings': account_results['monthly_meetings_by_segment'].get(segment, 0),
            'Weekly Hours': account_results['monthly_meetings_by_segment'].get(segment, 0) / 4.33 * account_results['time_per_meeting']
        })
    
    # Portfolio totals
    account_data.append({
        'Category': 'Portfolio Totals',
        'Metric': 'Total Existing Accounts',
        'Count': account_results['total_accounts'],
        'Percentage': '100.0%',
        'Monthly Meetings': sum(account_results['monthly_meetings_by_segment'].values()),
        'Weekly Hours': sum(account_results['monthly_meetings_by_segment'].values()) / 4.33 * account_results['time_per_meeting']
    })
    
    # New customer onboarding
    for meeting_type, count in account_results['onboarding_meetings'].items():
        account_data.append({
            'Category': 'New Customer Onboarding',
            'Metric': f'{meeting_type.replace("_", " ").title()}',
            'Count': account_results['new_logos_quarterly'],
            'Percentage': f'{onboarding_params[f"""{meeting_type.split("_")[0]}_onboarding_percentage"""]}%',
            'Monthly Meetings': count,
            'Weekly Hours': count / 4.33 * account_results['time_per_meeting']
        })
    
    # Combined totals
    account_data.append({
        'Category': 'Combined Totals',
        'Metric': 'Total Account Management',
        'Count': account_results['total_accounts'] + account_results['new_logos_quarterly'],
        'Percentage': '100.0%',
        'Monthly Meetings': account_results['total_monthly_meetings'],
        'Weekly Hours': account_results['weekly_account_hours']
    })
    
    # Load distribution
    account_data.append({
        'Category': 'Load Distribution',
        'Metric': 'Per SE (IC + Director IC time)',
        'Count': distribution_results['accounts_per_se'],
        'Percentage': f'{distribution_results["hours_per_se_weekly"]/40*100:.1f}% capacity',
        'Monthly Meetings': distribution_results['meetings_per_se_weekly'] * 4.33,
        'Weekly Hours': distribution_results['hours_per_se_weekly']
    })
    
    df = pd.DataFrame(account_data)
    
    print("\n📊 Account Management Summary Table")
    print("=" * 40)
    
    # Format numeric columns for display
    display_df = df.copy()
    numeric_columns = ['Count', 'Monthly Meetings', 'Weekly Hours']
    for col in numeric_columns:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: f'{x:.1f}' if isinstance(x, (int, float)) else str(x))
    
    print(display_df.to_string(index=False))
    
    return df

def validate_account_management_calculations(account_results, distribution_results):
    """
    Validate account management calculations for reasonableness and consistency.
    
    Parameters:
    - account_results (dict): Results from run_account_management_calculations()
    - distribution_results (dict): Results from calculate_account_load_distribution()
    
    Returns:
    - tuple: (is_valid, validation_messages)
    """
    
    print("\n🔍 Account Management Validation")
    print("=" * 35)
    
    validation_messages = []
    warnings = []
    is_valid = True
    
    # Validate mathematical consistency
    expected_onboarding_total = sum(account_results['onboarding_meetings'].values())
    calculated_onboarding = account_results['total_monthly_meetings'] - sum(account_results['monthly_meetings_by_segment'].values())
    
    if abs(expected_onboarding_total - calculated_onboarding) > 0.01:
        validation_messages.append(f"❌ Onboarding calculation inconsistency: {expected_onboarding_total:.2f} vs {calculated_onboarding:.2f}")
        is_valid = False
    else:
        validation_messages.append("✅ Onboarding calculations mathematically consistent")
    
    # Validate time calculations
    expected_weekly_hours = account_results['total_monthly_meetings'] / 4.33 * account_results['time_per_meeting']
    actual_weekly_hours = account_results['weekly_account_hours']
    
    if abs(expected_weekly_hours - actual_weekly_hours) > 0.01:
        validation_messages.append(f"❌ Time calculation inconsistency: {expected_weekly_hours:.2f} vs {actual_weekly_hours:.2f}")
        is_valid = False
    else:
        validation_messages.append("✅ Time calculations mathematically consistent")
    
    # Validate reasonable values
    if account_results['total_accounts'] == 0:
        warnings.append("No accounts in portfolio - unusual scenario")
    
    if account_results['weekly_account_hours'] > 40:
        warnings.append(f"Account management exceeds full-time capacity: {account_results['weekly_account_hours']:.1f} hrs/week")
    
    if distribution_results['accounts_per_se'] > 100:
        warnings.append(f"Extremely high account load per SE: {distribution_results['accounts_per_se']:.1f} accounts")
    
    # Check onboarding percentages
    total_onboarding_pct = (onboarding_params['monthly_onboarding_percentage'] + 
                           onboarding_params['quarterly_onboarding_percentage'])
    if total_onboarding_pct > 100:
        validation_messages.append(f"❌ Onboarding percentages exceed 100%: {total_onboarding_pct}%")
        is_valid = False
    elif total_onboarding_pct == 0:
        warnings.append("No new customer onboarding meetings configured")
    
    # Display results
    for message in validation_messages:
        print(message)
    
    if warnings:
        print(f"\n⚠️  Warnings:")
        for warning in warnings:
            print(f"   • {warning}")
    
    print(f"\n📊 Validation Summary:")
    if is_valid:
        print("✅ Account management calculations validated successfully")
        if warnings:
            print(f"⚠️  {len(warnings)} warnings noted")
    else:
        print("❌ Account management validation failed - check calculations")
    
    return is_valid, validation_messages + warnings

def analyze_account_portfolio_balance():
    """Analyze the balance and health of the account portfolio."""
    
    print("\n⚖️  Account Portfolio Balance Analysis")
    print("=" * 42)
    
    total_accounts = sum(account_params.values())
    
    if total_accounts == 0:
        print("❌ No accounts in portfolio")
        return {}
    
    # Calculate portfolio distribution
    strategic_pct = account_params['strategic_accounts'] / total_accounts * 100
    protect_pct = account_params['protect_accounts'] / total_accounts * 100  
    growth_pct = account_params['growth_accounts'] / total_accounts * 100
    retain_pct = account_params['retain_accounts'] / total_accounts * 100
    
    print(f"📊 Portfolio Distribution Analysis:")
    print(f"   • Strategic (High Value): {strategic_pct:.1f}% - Target: 5-15%")
    print(f"   • Protect (At Risk): {protect_pct:.1f}% - Target: 10-20%")
    print(f"   • Growth (Expansion): {growth_pct:.1f}% - Target: 20-40%")
    print(f"   • Retain (Maintenance): {retain_pct:.1f}% - Target: 40-60%")
    
    # Portfolio health assessment
    health_issues = []
    recommendations = []
    
    if strategic_pct > 20:
        health_issues.append(f"Too many strategic accounts ({strategic_pct:.1f}%)")
        recommendations.append("Consider graduated approach to reduce strategic meeting frequency")
    elif strategic_pct < 2:
        health_issues.append(f"Very few strategic accounts ({strategic_pct:.1f}%)")
        recommendations.append("Identify key accounts for strategic relationship investment")
    
    if protect_pct > 25:
        health_issues.append(f"High percentage of at-risk accounts ({protect_pct:.1f}%)")
        recommendations.append("Focus on retention strategies to move accounts to Growth")
    
    if retain_pct > 70:
        health_issues.append(f"Portfolio heavily skewed to maintenance ({retain_pct:.1f}%)")
        recommendations.append("Identify growth opportunities within retain segment")
    
    if health_issues:
        print(f"\n⚠️  Portfolio Health Issues:")
        for issue in health_issues:
            print(f"   • {issue}")
        print(f"\n💡 Recommendations:")
        for rec in recommendations:
            print(f"   • {rec}")
    else:
        print(f"\n✅ Portfolio distribution appears well-balanced")
    
    balance_analysis = {
        'distribution': {
            'strategic': strategic_pct,
            'protect': protect_pct,
            'growth': growth_pct,
            'retain': retain_pct
        },
        'health_issues': health_issues,
        'recommendations': recommendations
    }
    
    return balance_analysis

# ===================================
# MAIN EXECUTION
# ===================================

# Run the complete account management calculation flow
account_results = run_account_management_calculations()

# Calculate load distribution
distribution_results = calculate_account_load_distribution(account_results)

# Create summary DataFrame
account_df = create_account_management_dataframe(account_results, distribution_results)

# Validate calculations
is_valid, validation_messages = validate_account_management_calculations(account_results, distribution_results)

# Analyze portfolio balance
balance_analysis = analyze_account_portfolio_balance()

# Store results for use in other sections
ACCOUNT_RESULTS = {
    'account_summary': account_results,
    'distribution_analysis': distribution_results,
    'account_df': account_df,
    'validation_status': is_valid,
    'validation_messages': validation_messages,
    'balance_analysis': balance_analysis
}

if is_valid:
    print("\n✅ Account Management Logic completed successfully")
    print("📝 Ready for Step 6: Strategic Activities Logic")
else:
    print("\n❌ Account management calculations need review before proceeding")
    print("📝 Please address validation issues")

---

# 5. Strategic Activities Logic {#strategic-activities}

*Strategic time allocation and distribution between ICs and Directors*

In [ ]:
# Strategic Activities Logic Engine
# Strategic time allocation and distribution between ICs and Directors

# ===================================
# STRATEGIC ACTIVITIES CALCULATION ENGINE
# ===================================

def run_strategic_activities_calculations():
    """
    Execute complete strategic activities calculation flow for team allocation.
    
    Returns:
    - dict: Comprehensive strategic activities analysis results
    """
    print("🎯 Running Strategic Activities Calculations")
    print("=" * 50)
    
    # Step 1: Strategic Activity Inventory
    print("\n📋 Step 1: Strategic Activity Inventory")
    print("-" * 38)
    
    # Define strategic activities with their time ranges
    strategic_activities = {
        'customer_zero': {
            'min_hours': strategic_params['customer_zero_hours_min'],
            'max_hours': strategic_params['customer_zero_hours_max'],
            'description': 'Customer zero engagement and advocacy'
        },
        'content_creation': {
            'hours': strategic_params['content_creation_hours'],
            'description': 'Blog posts, documentation, technical content'
        },
        'sales_enablement': {
            'hours': strategic_params['sales_enablement_hours'],
            'description': 'Sales team training and support materials'
        },
        'developer_advocacy': {
            'min_hours': strategic_params['developer_advocacy_hours_min'],
            'max_hours': strategic_params['developer_advocacy_hours_max'],
            'description': 'Community engagement and developer relations'
        },
        'asset_development': {
            'hours': strategic_params['asset_development_hours_avg'],
            'description': 'Demo environments, proof-of-concepts, tools'
        }
    }
    
    print(f"🔍 Strategic Activity Portfolio:")
    total_min_hours = 0
    total_max_hours = 0
    
    for activity, details in strategic_activities.items():
        if 'min_hours' in details and 'max_hours' in details:
            # Variable range activity
            min_h = details['min_hours']
            max_h = details['max_hours']
            avg_h = (min_h + max_h) / 2
            total_min_hours += min_h
            total_max_hours += max_h
            print(f"   • {activity.replace('_', ' ').title()}: {min_h}-{max_h} hrs/week (avg: {avg_h:.1f})")
            print(f"     {details['description']}")
        else:
            # Fixed hours activity
            hours = details['hours']
            total_min_hours += hours
            total_max_hours += hours
            print(f"   • {activity.replace('_', ' ').title()}: {hours} hrs/week")
            print(f"     {details['description']}")
    
    print(f"\n📊 Total Strategic Time Range: {total_min_hours:.1f}-{total_max_hours:.1f} hrs/week")
    avg_strategic_hours = (total_min_hours + total_max_hours) / 2
    print(f"📈 Average Strategic Time: {avg_strategic_hours:.1f} hrs/week")
    
    # Step 2: Team Structure Analysis
    print("\n👥 Step 2: Team Structure Analysis")
    print("-" * 35)
    
    total_ics = staffing_params['num_ic_ses']
    total_directors = staffing_params['num_directors']
    director_ic_percentage = staffing_params['director_ic_percentage']
    ic_strategic_percentage = strategic_params['ic_strategic_percentage']
    
    print(f"🏗️  Team Composition:")
    print(f"   • IC SEs: {total_ics}")
    print(f"   • Directors: {total_directors}")
    print(f"   • Total Team: {total_ics + total_directors}")
    
    print(f"\n⚖️  Role Distribution:")
    print(f"   • Director IC Time: {director_ic_percentage}%")
    print(f"   • Director Strategic Time: {100 - director_ic_percentage}%")
    print(f"   • IC Strategic Focus: {ic_strategic_percentage}%")
    print(f"   • Director Strategic Focus: {100 - ic_strategic_percentage}%")
    
    # Calculate effective capacities
    director_strategic_capacity = 100 - director_ic_percentage
    effective_ic_capacity = total_ics + (total_directors * director_ic_percentage / 100)
    effective_strategic_capacity = total_ics + total_directors  # Both can do strategic work
    
    print(f"\n📊 Effective Capacities:")
    print(f"   • IC Work Capacity: {effective_ic_capacity:.1f} FTE")
    print(f"   • Strategic Work Capacity: {effective_strategic_capacity:.1f} FTE")
    print(f"   • Director Strategic Capacity: {director_strategic_capacity}% of director time")
    
    # Step 3: Strategic Work Distribution
    print("\n🎯 Step 3: Strategic Work Distribution")
    print("-" * 38)
    
    # Use helper function to calculate strategic workload distribution
    strategic_distribution = calculate_strategic_workload(strategic_params, staffing_params)
    
    print(f"📋 Strategic Work Allocation:")
    print(f"   • Total Strategic Hours: {avg_strategic_hours:.1f} hrs/week")
    print(f"   • IC Allocation: {ic_strategic_percentage}% = {strategic_distribution['total_ic_strategic_hours']:.1f} hrs/week")
    print(f"   • Director Allocation: {100-ic_strategic_percentage}% = {strategic_distribution['total_director_strategic_hours']:.1f} hrs/week")
    
    print(f"\n👤 Per-Person Strategic Load:")
    if total_ics > 0:
        print(f"   • Per IC: {strategic_distribution['ic_strategic_hours_per_se']:.1f} hrs/week")
        print(f"   • IC Strategic Utilization: {strategic_distribution['ic_strategic_hours_per_se']/40*100:.1f}% of capacity")
    else:
        print(f"   • No ICs available for strategic work")
    
    if total_directors > 0:
        print(f"   • Per Director (Strategic): {strategic_distribution['director_strategic_hours_per_director']:.1f} hrs/week")
        print(f"   • Director Strategic Utilization: {strategic_distribution['director_strategic_hours_per_director']/(40*director_strategic_capacity/100)*100:.1f}% of strategic capacity")
    else:
        print(f"   • No Directors available for strategic work")
    
    # Step 4: Activity-Specific Allocation
    print("\n📝 Step 4: Activity-Specific Allocation")
    print("-" * 40)
    
    # Distribute total strategic hours across activities proportionally
    activity_allocation = {}
    
    for activity, details in strategic_activities.items():
        if 'min_hours' in details and 'max_hours' in details:
            # Use average for variable activities
            activity_hours = (details['min_hours'] + details['max_hours']) / 2
        else:
            activity_hours = details['hours']
        
        # Calculate proportion of total strategic time
        proportion = activity_hours / avg_strategic_hours
        
        # Allocate to ICs and Directors based on strategic percentage
        ic_hours = strategic_distribution['total_ic_strategic_hours'] * proportion
        director_hours = strategic_distribution['total_director_strategic_hours'] * proportion
        
        activity_allocation[activity] = {
            'total_hours': activity_hours,
            'ic_hours': ic_hours,
            'director_hours': director_hours,
            'proportion': proportion
        }
        
        print(f"🎯 {activity.replace('_', ' ').title()}:")
        print(f"   • Total: {activity_hours:.1f} hrs/week ({proportion:.1%} of strategic time)")
        print(f"   • IC Allocation: {ic_hours:.1f} hrs/week")
        print(f"   • Director Allocation: {director_hours:.1f} hrs/week")
    
    # Step 5: Strategic Activities Summary
    strategic_summary = {
        'strategic_activities': strategic_activities,
        'total_min_hours': total_min_hours,
        'total_max_hours': total_max_hours,
        'avg_strategic_hours': avg_strategic_hours,
        'team_structure': {
            'total_ics': total_ics,
            'total_directors': total_directors,
            'director_ic_percentage': director_ic_percentage,
            'ic_strategic_percentage': ic_strategic_percentage,
            'effective_ic_capacity': effective_ic_capacity,
            'effective_strategic_capacity': effective_strategic_capacity
        },
        'strategic_distribution': strategic_distribution,
        'activity_allocation': activity_allocation
    }
    
    return strategic_summary

def calculate_strategic_capacity_impact(strategic_results):
    """
    Calculate how strategic activities impact overall SE capacity.
    
    Parameters:
    - strategic_results (dict): Results from run_strategic_activities_calculations()
    
    Returns:
    - dict: Strategic capacity impact analysis
    """
    
    print("\n📊 Strategic Capacity Impact Analysis")
    print("=" * 40)
    
    # Calculate remaining capacity after strategic activities
    total_ics = strategic_results['team_structure']['total_ics']
    total_directors = strategic_results['team_structure']['total_directors']
    director_ic_percentage = strategic_results['team_structure']['director_ic_percentage']
    
    # Total weekly capacity
    total_ic_capacity = total_ics * 40  # 40 hours per week
    total_director_ic_capacity = total_directors * 40 * (director_ic_percentage / 100)
    total_director_strategic_capacity = total_directors * 40 * ((100 - director_ic_percentage) / 100)
    
    # Strategic time allocation
    ic_strategic_hours = strategic_results['strategic_distribution']['total_ic_strategic_hours']
    director_strategic_hours = strategic_results['strategic_distribution']['total_director_strategic_hours']
    
    # Remaining capacity for pipeline activities
    remaining_ic_capacity = total_ic_capacity - ic_strategic_hours
    remaining_director_ic_capacity = total_director_ic_capacity  # Directors do IC work separate from strategic
    total_remaining_pipeline_capacity = remaining_ic_capacity + remaining_director_ic_capacity
    
    print(f"⚡ Total Team Capacity Breakdown:")
    print(f"   • IC Capacity: {total_ic_capacity:.1f} hrs/week")
    print(f"   • Director IC Capacity: {total_director_ic_capacity:.1f} hrs/week")
    print(f"   • Director Strategic Capacity: {total_director_strategic_capacity:.1f} hrs/week")
    print(f"   • Total Pipeline Capacity: {total_ic_capacity + total_director_ic_capacity:.1f} hrs/week")
    
    print(f"\n🎯 Strategic Time Allocation:")
    print(f"   • IC Strategic Hours: {ic_strategic_hours:.1f} hrs/week ({ic_strategic_hours/total_ic_capacity*100:.1f}% of IC capacity)")
    print(f"   • Director Strategic Hours: {director_strategic_hours:.1f} hrs/week ({director_strategic_hours/total_director_strategic_capacity*100:.1f}% of director strategic capacity)")
    print(f"   • Total Strategic Hours: {ic_strategic_hours + director_strategic_hours:.1f} hrs/week")
    
    print(f"\n🔄 Remaining Pipeline Capacity:")
    print(f"   • IC Pipeline Capacity: {remaining_ic_capacity:.1f} hrs/week")
    print(f"   • Director IC Pipeline Capacity: {remaining_director_ic_capacity:.1f} hrs/week")
    print(f"   • Total Pipeline Capacity: {total_remaining_pipeline_capacity:.1f} hrs/week")
    
    # Calculate utilization rates
    strategic_utilization = {
        'ic_strategic_utilization': ic_strategic_hours / total_ic_capacity * 100 if total_ic_capacity > 0 else 0,
        'director_strategic_utilization': director_strategic_hours / total_director_strategic_capacity * 100 if total_director_strategic_capacity > 0 else 0,
        'overall_strategic_utilization': (ic_strategic_hours + director_strategic_hours) / (total_ic_capacity + total_director_strategic_capacity) * 100 if (total_ic_capacity + total_director_strategic_capacity) > 0 else 0
    }
    
    print(f"\n📈 Strategic Utilization Rates:")
    print(f"   • IC Strategic Utilization: {strategic_utilization['ic_strategic_utilization']:.1f}%")
    print(f"   • Director Strategic Utilization: {strategic_utilization['director_strategic_utilization']:.1f}%")
    print(f"   • Overall Strategic Utilization: {strategic_utilization['overall_strategic_utilization']:.1f}%")
    
    capacity_impact = {
        'total_capacities': {
            'total_ic_capacity': total_ic_capacity,
            'total_director_ic_capacity': total_director_ic_capacity,
            'total_director_strategic_capacity': total_director_strategic_capacity,
            'total_pipeline_capacity': total_ic_capacity + total_director_ic_capacity
        },
        'strategic_allocation': {
            'ic_strategic_hours': ic_strategic_hours,
            'director_strategic_hours': director_strategic_hours,
            'total_strategic_hours': ic_strategic_hours + director_strategic_hours
        },
        'remaining_pipeline_capacity': {
            'remaining_ic_capacity': remaining_ic_capacity,
            'remaining_director_ic_capacity': remaining_director_ic_capacity,
            'total_remaining_pipeline_capacity': total_remaining_pipeline_capacity
        },
        'utilization_rates': strategic_utilization
    }
    
    return capacity_impact

def create_strategic_activities_dataframe(strategic_results, capacity_impact):
    """
    Create comprehensive DataFrame summarizing strategic activities calculations.
    
    Parameters:
    - strategic_results (dict): Results from run_strategic_activities_calculations()
    - capacity_impact (dict): Results from calculate_strategic_capacity_impact()
    
    Returns:
    - pd.DataFrame: Structured strategic activities data
    """
    
    strategic_data = []
    
    # Strategic activities breakdown
    for activity, allocation in strategic_results['activity_allocation'].items():
        activity_name = activity.replace('_', ' ').title()
        strategic_data.append({
            'Category': 'Strategic Activities',
            'Activity': activity_name,
            'Total Hours/Week': allocation['total_hours'],
            'IC Hours/Week': allocation['ic_hours'],
            'Director Hours/Week': allocation['director_hours'],
            'Proportion of Strategic Time': f"{allocation['proportion']:.1%}"
        })
    
    # Strategic totals
    strategic_data.append({
        'Category': 'Strategic Totals',
        'Activity': 'All Strategic Activities',
        'Total Hours/Week': strategic_results['avg_strategic_hours'],
        'IC Hours/Week': strategic_results['strategic_distribution']['total_ic_strategic_hours'],
        'Director Hours/Week': strategic_results['strategic_distribution']['total_director_strategic_hours'],
        'Proportion of Strategic Time': '100.0%'
    })
    
    # Capacity impact
    strategic_data.append({
        'Category': 'Capacity Impact',
        'Activity': 'Total Team Capacity',
        'Total Hours/Week': capacity_impact['total_capacities']['total_ic_capacity'] + capacity_impact['total_capacities']['total_director_strategic_capacity'],
        'IC Hours/Week': capacity_impact['total_capacities']['total_ic_capacity'],
        'Director Hours/Week': capacity_impact['total_capacities']['total_director_strategic_capacity'],
        'Proportion of Strategic Time': 'N/A'
    })
    
    strategic_data.append({
        'Category': 'Capacity Impact',
        'Activity': 'Remaining Pipeline Capacity',
        'Total Hours/Week': capacity_impact['remaining_pipeline_capacity']['total_remaining_pipeline_capacity'],
        'IC Hours/Week': capacity_impact['remaining_pipeline_capacity']['remaining_ic_capacity'],
        'Director Hours/Week': capacity_impact['remaining_pipeline_capacity']['remaining_director_ic_capacity'],
        'Proportion of Strategic Time': 'N/A'
    })
    
    # Utilization rates
    strategic_data.append({
        'Category': 'Utilization Rates',
        'Activity': 'Strategic Utilization',
        'Total Hours/Week': f"{capacity_impact['utilization_rates']['overall_strategic_utilization']:.1f}%",
        'IC Hours/Week': f"{capacity_impact['utilization_rates']['ic_strategic_utilization']:.1f}%",
        'Director Hours/Week': f"{capacity_impact['utilization_rates']['director_strategic_utilization']:.1f}%",
        'Proportion of Strategic Time': 'N/A'
    })
    
    df = pd.DataFrame(strategic_data)
    
    print("\n📊 Strategic Activities Summary Table")
    print("=" * 42)
    
    # Format numeric columns for display
    display_df = df.copy()
    numeric_columns = ['Total Hours/Week', 'IC Hours/Week', 'Director Hours/Week']
    for col in numeric_columns:
        if col in display_df.columns:
            display_df[col] = display_df[col].apply(lambda x: f'{x:.1f}' if isinstance(x, (int, float)) else str(x))
    
    print(display_df.to_string(index=False))
    
    return df

def validate_strategic_activities_calculations(strategic_results, capacity_impact):
    """
    Validate strategic activities calculations for reasonableness and consistency.
    
    Parameters:
    - strategic_results (dict): Results from run_strategic_activities_calculations()
    - capacity_impact (dict): Results from calculate_strategic_capacity_impact()
    
    Returns:
    - tuple: (is_valid, validation_messages)
    """
    
    print("\n🔍 Strategic Activities Validation")
    print("=" * 37)
    
    validation_messages = []
    warnings = []
    is_valid = True
    
    # Validate mathematical consistency
    expected_total_strategic = strategic_results['strategic_distribution']['total_ic_strategic_hours'] + strategic_results['strategic_distribution']['total_director_strategic_hours']
    actual_total_strategic = strategic_results['avg_strategic_hours']
    
    if abs(expected_total_strategic - actual_total_strategic) > 0.01:
        validation_messages.append(f"❌ Strategic hours calculation inconsistency: {expected_total_strategic:.2f} vs {actual_total_strategic:.2f}")
        is_valid = False
    else:
        validation_messages.append("✅ Strategic hours calculations mathematically consistent")
    
    # Validate activity allocation sums to total
    total_activity_hours = sum(allocation['total_hours'] for allocation in strategic_results['activity_allocation'].values())
    if abs(total_activity_hours - strategic_results['avg_strategic_hours']) > 0.01:
        validation_messages.append(f"❌ Activity allocation inconsistency: {total_activity_hours:.2f} vs {strategic_results['avg_strategic_hours']:.2f}")
        is_valid = False
    else:
        validation_messages.append("✅ Activity allocation sums correctly")
    
    # Validate capacity calculations
    total_team_capacity = capacity_impact['total_capacities']['total_ic_capacity'] + capacity_impact['total_capacities']['total_director_strategic_capacity']
    if total_team_capacity <= 0:
        validation_messages.append("❌ No team capacity available")
        is_valid = False
    else:
        validation_messages.append("✅ Team capacity calculations valid")
    
    # Check for reasonable utilization rates
    ic_utilization = capacity_impact['utilization_rates']['ic_strategic_utilization']
    director_utilization = capacity_impact['utilization_rates']['director_strategic_utilization']
    
    if ic_utilization > 50:
        warnings.append(f"High IC strategic utilization: {ic_utilization:.1f}%")
    
    if director_utilization > 80:
        warnings.append(f"High director strategic utilization: {director_utilization:.1f}%")
    
    # Check remaining pipeline capacity
    remaining_capacity = capacity_impact['remaining_pipeline_capacity']['total_remaining_pipeline_capacity']
    if remaining_capacity < 20:  # Less than 20 hours per week for pipeline work
        warnings.append(f"Very low remaining pipeline capacity: {remaining_capacity:.1f} hrs/week")
    
    # Validate percentage parameters
    ic_strategic_pct = strategic_params['ic_strategic_percentage']
    if not (0 <= ic_strategic_pct <= 100):
        validation_messages.append(f"❌ Invalid IC strategic percentage: {ic_strategic_pct}%")
        is_valid = False
    
    # Display results
    for message in validation_messages:
        print(message)
    
    if warnings:
        print(f"\n⚠️  Warnings:")
        for warning in warnings:
            print(f"   • {warning}")
    
    print(f"\n📊 Validation Summary:")
    if is_valid:
        print("✅ Strategic activities calculations validated successfully")
        if warnings:
            print(f"⚠️  {len(warnings)} warnings noted")
    else:
        print("❌ Strategic activities validation failed - check calculations")
    
    return is_valid, validation_messages + warnings

def analyze_strategic_balance():
    """Analyze the balance and appropriateness of strategic activity allocation."""
    
    print("\n⚖️  Strategic Activity Balance Analysis")
    print("=" * 42)
    
    total_strategic = strategic_params['customer_zero_hours_min'] + strategic_params['customer_zero_hours_max']
    total_strategic += strategic_params['content_creation_hours'] * 2  # Weight content creation
    total_strategic += strategic_params['sales_enablement_hours']
    total_strategic += strategic_params['developer_advocacy_hours_min'] + strategic_params['developer_advocacy_hours_max']
    total_strategic += strategic_params['asset_development_hours_avg']
    
    if total_strategic == 0:
        print("❌ No strategic activities configured")
        return {}
    
    # Calculate proportions
    customer_zero_avg = (strategic_params['customer_zero_hours_min'] + strategic_params['customer_zero_hours_max']) / 2
    developer_advocacy_avg = (strategic_params['developer_advocacy_hours_min'] + strategic_params['developer_advocacy_hours_max']) / 2
    
    activities_total = (customer_zero_avg + strategic_params['content_creation_hours'] + 
                       strategic_params['sales_enablement_hours'] + developer_advocacy_avg + 
                       strategic_params['asset_development_hours_avg'])
    
    proportions = {
        'customer_zero': customer_zero_avg / activities_total * 100,
        'content_creation': strategic_params['content_creation_hours'] / activities_total * 100,
        'sales_enablement': strategic_params['sales_enablement_hours'] / activities_total * 100,
        'developer_advocacy': developer_advocacy_avg / activities_total * 100,
        'asset_development': strategic_params['asset_development_hours_avg'] / activities_total * 100
    }
    
    print(f"📊 Strategic Activity Distribution:")
    for activity, percentage in proportions.items():
        print(f"   • {activity.replace('_', ' ').title()}: {percentage:.1f}%")
    
    # Balance assessment
    balance_issues = []
    recommendations = []
    
    if proportions['customer_zero'] > 40:
        balance_issues.append(f"Customer zero dominates strategic time ({proportions['customer_zero']:.1f}%)")
        recommendations.append("Consider balancing with other strategic activities")
    elif proportions['customer_zero'] < 15:
        balance_issues.append(f"Low customer zero investment ({proportions['customer_zero']:.1f}%)")
        recommendations.append("Consider increasing customer advocacy efforts")
    
    if proportions['content_creation'] < 10:
        balance_issues.append(f"Low content creation investment ({proportions['content_creation']:.1f}%)")
        recommendations.append("Consider increasing technical content development")
    
    if proportions['developer_advocacy'] < 5:
        balance_issues.append(f"Minimal developer advocacy ({proportions['developer_advocacy']:.1f}%)")
        recommendations.append("Consider community engagement opportunities")
    
    if activities_total > 15:
        balance_issues.append(f"High total strategic commitment ({activities_total:.1f} hrs/week)")
        recommendations.append("May limit pipeline capacity - consider prioritization")
    elif activities_total < 5:
        balance_issues.append(f"Low strategic activity investment ({activities_total:.1f} hrs/week)")
        recommendations.append("Consider increasing strategic initiatives for long-term growth")
    
    if balance_issues:
        print(f"\n⚠️  Strategic Balance Issues:")
        for issue in balance_issues:
            print(f"   • {issue}")
        print(f"\n💡 Recommendations:")
        for rec in recommendations:
            print(f"   • {rec}")
    else:
        print(f"\n✅ Strategic activity balance appears appropriate")
    
    balance_analysis = {
        'activity_proportions': proportions,
        'total_strategic_hours': activities_total,
        'balance_issues': balance_issues,
        'recommendations': recommendations
    }
    
    return balance_analysis

# ===================================
# MAIN EXECUTION
# ===================================

# Run the complete strategic activities calculation flow
strategic_results = run_strategic_activities_calculations()

# Calculate capacity impact
capacity_impact = calculate_strategic_capacity_impact(strategic_results)

# Create summary DataFrame
strategic_df = create_strategic_activities_dataframe(strategic_results, capacity_impact)

# Validate calculations
is_valid, validation_messages = validate_strategic_activities_calculations(strategic_results, capacity_impact)

# Analyze strategic balance
balance_analysis = analyze_strategic_balance()

# Store results for use in other sections
STRATEGIC_RESULTS = {
    'strategic_summary': strategic_results,
    'capacity_impact': capacity_impact,
    'strategic_df': strategic_df,
    'validation_status': is_valid,
    'validation_messages': validation_messages,
    'balance_analysis': balance_analysis
}

if is_valid:
    print("\n✅ Strategic Activities Logic completed successfully")
    print("📝 Ready for Step 7: Basic Widgets Implementation")
    print("\n🎉 Phase 2: Business Logic Implementation COMPLETE!")
else:
    print("\n❌ Strategic activities calculations need review before proceeding")
    print("📝 Please address validation issues")

---

# 6. Interactive Parameter Controls {#interactive-controls}

*Widget-based interface for parameter input and real-time updates*

In [ ]:
# Placeholder for interactive controls
# Will be implemented in Steps 7-9

print("🎛️ Interactive controls section ready for implementation")

---

# 7. Real-time Calculations {#real-time-calculations}

*Live calculation updates based on parameter changes*

In [ ]:
# Placeholder for real-time calculations
# Will be implemented in Step 9

print("⚡ Real-time calculations section ready for implementation")

---

# 8. Core Visualizations {#core-visualizations}

*Essential charts for utilization analysis and capacity planning*

In [ ]:
# Core Visualizations Engine
# Essential charts for utilization analysis and capacity planning

# ===================================
# VISUALIZATION HELPER FUNCTIONS
# ===================================

def setup_chart_style():
    """Configure consistent styling for all charts."""
    plt.style.use('default')
    sns.set_style("whitegrid")
    sns.set_palette("Set2")
    
    # Set consistent figure parameters
    plt.rcParams.update({
        'figure.figsize': (12, 8),
        'font.size': 11,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 11,
        'figure.facecolor': 'white',
        'axes.facecolor': 'white'
    })

def format_currency(value):
    """Format currency values for display."""
    if value >= 1000000:
        return f"${value/1000000:.1f}M"
    elif value >= 1000:
        return f"${value/1000:.0f}K"
    else:
        return f"${value:.0f}"

def format_hours(hours):
    """Format hours for display with appropriate precision."""
    if hours < 0.1:
        return "< 0.1h"
    elif hours < 1:
        return f"{hours:.1f}h"
    else:
        return f"{hours:.1f}h"

# ===================================
# CORE VISUALIZATION FUNCTIONS
# ===================================

def create_capacity_utilization_chart():
    """Create a comprehensive capacity utilization chart."""
    
    print("📊 Creating Capacity Utilization Chart")
    print("=" * 40)
    
    # Gather data from previous calculations
    if 'PIPELINE_RESULTS' not in globals():
        print("❌ Pipeline results not available. Please run pipeline calculations first.")
        return None
    
    if 'ACCOUNT_RESULTS' not in globals():
        print("❌ Account results not available. Please run account management calculations first.")
        return None
        
    if 'STRATEGIC_RESULTS' not in globals():
        print("❌ Strategic results not available. Please run strategic activities calculations first.")
        return None
    
    # Extract data
    pipeline_hours = PIPELINE_RESULTS['pipeline_summary']['total_pipeline_hours']
    account_hours = ACCOUNT_RESULTS['account_summary']['weekly_account_hours']
    strategic_hours = STRATEGIC_RESULTS['strategic_summary']['avg_strategic_hours']
    fixed_hours = sum(fixed_activity_params.values())
    
    # Calculate team capacity
    total_ics = staffing_params['num_ic_ses']
    total_directors = staffing_params['num_directors']
    director_ic_percentage = staffing_params['director_ic_percentage']
    
    # Total capacity calculation
    total_ic_capacity = total_ics * 40
    total_director_capacity = total_directors * 40
    director_ic_capacity = total_directors * 40 * (director_ic_percentage / 100)
    director_strategic_capacity = total_directors * 40 * ((100 - director_ic_percentage) / 100)
    
    total_capacity = total_ic_capacity + total_director_capacity
    total_operational_capacity = total_ic_capacity + director_ic_capacity
    
    # Create figure with subplots
    setup_chart_style()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('SE Team Capacity Utilization Analysis', fontsize=16, fontweight='bold')
    
    # Chart 1: Overall Utilization Breakdown
    activities = ['Pipeline\nActivities', 'Account\nManagement', 'Strategic\nActivities', 'Fixed\nActivities', 'Available\nCapacity']
    hours = [pipeline_hours, account_hours, strategic_hours, fixed_hours, 
             max(0, total_operational_capacity - (pipeline_hours + account_hours + strategic_hours + fixed_hours))]
    
    colors = sns.color_palette("Set2", len(activities))
    bars1 = ax1.bar(activities, hours, color=colors)
    ax1.set_title('Weekly Hours by Activity Type', fontweight='bold')
    ax1.set_ylabel('Hours per Week')
    ax1.set_ylim(0, max(total_operational_capacity * 1.1, max(hours) * 1.1))
    
    # Add capacity line
    ax1.axhline(y=total_operational_capacity, color='red', linestyle='--', alpha=0.7, 
                label=f'Total Capacity ({total_operational_capacity:.1f}h)')
    ax1.legend()
    
    # Add value labels on bars
    for bar, hour in zip(bars1, hours):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(hour)}',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 2: Utilization Percentages
    total_used = pipeline_hours + account_hours + strategic_hours + fixed_hours
    utilization_pcts = [h/total_operational_capacity*100 for h in hours[:-1]]  # Exclude available capacity
    utilization_pcts.append(max(0, (total_operational_capacity - total_used)/total_operational_capacity*100))
    
    wedges, texts, autotexts = ax2.pie(hours, labels=activities, autopct='%1.1f%%', 
                                       colors=colors, startangle=90)
    ax2.set_title('Capacity Utilization Distribution', fontweight='bold')
    
    # Chart 3: Individual vs Team Capacity
    roles = ['IC SEs', 'Directors\n(IC Work)', 'Directors\n(Strategic)', 'Combined\nOperational']
    capacities = [total_ic_capacity, director_ic_capacity, director_strategic_capacity, total_operational_capacity]
    
    bars3 = ax3.bar(roles, capacities, color=sns.color_palette("viridis", len(roles)))
    ax3.set_title('Team Capacity Breakdown', fontweight='bold')
    ax3.set_ylabel('Hours per Week')
    
    # Add value labels
    for bar, capacity in zip(bars3, capacities):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(capacity)}',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 4: Capacity vs Demand Comparison
    demand_categories = ['Pipeline\n+ Account', 'Strategic', 'Fixed\nActivities', 'Total\nDemand']
    operational_demand = pipeline_hours + account_hours
    demand_values = [operational_demand, strategic_hours, fixed_hours, total_used]
    capacity_values = [total_operational_capacity, director_strategic_capacity + (strategic_hours * total_ics / (total_ics + total_directors) if (total_ics + total_directors) > 0 else 0), total_operational_capacity, total_capacity]
    
    x_pos = np.arange(len(demand_categories))
    width = 0.35
    
    bars4a = ax4.bar(x_pos - width/2, demand_values, width, label='Demand', color=sns.color_palette("Set1")[0])
    bars4b = ax4.bar(x_pos + width/2, capacity_values, width, label='Capacity', color=sns.color_palette("Set1")[1])
    
    ax4.set_title('Demand vs Capacity Analysis', fontweight='bold')
    ax4.set_ylabel('Hours per Week')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(demand_categories)
    ax4.legend()
    
    # Add value labels
    for bar in bars4a:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(height)}',
                ha='center', va='bottom', fontsize=9)
    
    for bar in bars4b:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(height)}',
                ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\n📈 Capacity Utilization Summary:")
    print(f"   • Total Team Capacity: {format_hours(total_capacity)}")
    print(f"   • Operational Capacity: {format_hours(total_operational_capacity)}")
    print(f"   • Total Demand: {format_hours(total_used)}")
    print(f"   • Utilization Rate: {total_used/total_operational_capacity*100:.1f}%")
    print(f"   • Available Capacity: {format_hours(max(0, total_operational_capacity - total_used))}")
    
    return fig

def create_pipeline_funnel_chart():
    """Create a visual representation of the sales pipeline funnel."""
    
    print("\n🔄 Creating Pipeline Funnel Chart")
    print("=" * 38)
    
    if 'PIPELINE_RESULTS' not in globals():
        print("❌ Pipeline results not available. Please run pipeline calculations first.")
        return None
    
    # Extract pipeline data
    pipeline_data = PIPELINE_RESULTS['pipeline_summary']
    weekly_activities = pipeline_data['weekly_activities']
    
    # Calculate quarterly numbers for more impactful visualization
    quarters_per_year = 4
    weekly_to_quarterly = 13
    
    # Funnel stages (quarterly)
    meetings_quarterly = weekly_activities['initial_meetings'] * weekly_to_quarterly
    opportunities_quarterly = meetings_quarterly * (conversion_params['opportunity_creation_rate'] / 100)
    demos_quarterly = opportunities_quarterly * (conversion_params['demo_conversion_rate'] / 100)
    evaluations_quarterly = demos_quarterly * (conversion_params['tech_eval_conversion_rate'] / 100)
    wins_quarterly = evaluations_quarterly * pipeline_data['weighted_win_rate']
    
    # Revenue calculation
    revenue_quarterly = wins_quarterly * revenue_params['average_selling_price']
    
    setup_chart_style()
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    fig.suptitle('Sales Pipeline Funnel Analysis', fontsize=16, fontweight='bold')
    
    # Chart 1: Funnel Visualization
    stages = ['Initial\nMeetings', 'Opportunities\nCreated', 'Demos\nCompleted', 'Technical\nEvaluations', 'Closed\nWins']
    values = [meetings_quarterly, opportunities_quarterly, demos_quarterly, evaluations_quarterly, wins_quarterly]
    
    # Create funnel effect with different colors
    colors = sns.color_palette("viridis", len(stages))
    bars = ax1.barh(stages, values, color=colors)
    
    ax1.set_title('Quarterly Pipeline Funnel', fontweight='bold')
    ax1.set_xlabel('Count per Quarter')
    
    # Add value labels
    for i, (bar, value) in enumerate(zip(bars, values)):
        width = bar.get_width()
        ax1.text(width + max(values) * 0.01, bar.get_y() + bar.get_height()/2,
                f'{value:.1f}',
                ha='left', va='center', fontweight='bold')
        
        # Add conversion rates
        if i > 0:
            conversion_rate = (value / values[i-1]) * 100 if values[i-1] > 0 else 0
            ax1.text(width/2, bar.get_y() + bar.get_height()/2,
                    f'{conversion_rate:.1f}%',
                    ha='center', va='center', color='white', fontweight='bold')
    
    # Chart 2: Time Investment by Stage
    time_labels = ['Meetings', 'Demos', 'Evaluations']
    time_values = [
        PIPELINE_RESULTS['pipeline_summary']['meeting_hours'],
        PIPELINE_RESULTS['pipeline_summary']['demo_hours'],
        sum(PIPELINE_RESULTS['pipeline_summary']['evaluation_hours'].values())
    ]
    
    # Create stacked bar for time investment
    colors_time = sns.color_palette("Set1", len(time_labels))
    bars2 = ax2.bar(time_labels, time_values, color=colors_time)
    
    ax2.set_title('Weekly Time Investment by Activity', fontweight='bold')
    ax2.set_ylabel('Hours per Week')
    
    # Add value labels
    for bar, value in zip(bars2, time_values):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(value)}',
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print funnel summary
    print(f"\n🎯 Pipeline Funnel Summary (Quarterly):")
    print(f"   • Initial Meetings: {meetings_quarterly:.1f}")
    print(f"   • Opportunities: {opportunities_quarterly:.1f} ({opportunities_quarterly/meetings_quarterly*100:.1f}% conversion)")
    print(f"   • Demos: {demos_quarterly:.1f} ({demos_quarterly/opportunities_quarterly*100:.1f}% conversion)")
    print(f"   • Evaluations: {evaluations_quarterly:.1f} ({evaluations_quarterly/demos_quarterly*100:.1f}% conversion)")
    print(f"   • Wins: {wins_quarterly:.1f} ({wins_quarterly/evaluations_quarterly*100:.1f}% conversion)")
    print(f"   • Revenue: {format_currency(revenue_quarterly)}")
    print(f"   • Overall Conversion: {wins_quarterly/meetings_quarterly*100:.1f}% (meetings to wins)")
    
    return fig

def create_account_management_chart():
    """Create visualizations for account management workload."""
    
    print("\n👥 Creating Account Management Chart")
    print("=" * 40)
    
    if 'ACCOUNT_RESULTS' not in globals():
        print("❌ Account results not available. Please run account management calculations first.")
        return None
    
    account_data = ACCOUNT_RESULTS['account_summary']
    
    setup_chart_style()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Account Management Analysis', fontsize=16, fontweight='bold')
    
    # Chart 1: Account Portfolio Distribution
    segments = list(account_data['account_distribution'].keys())
    segment_counts = list(account_data['account_distribution'].values())
    segment_labels = [seg.replace('_', ' ').title() for seg in segments]
    
    colors1 = sns.color_palette("Set3", len(segments))
    wedges, texts, autotexts = ax1.pie(segment_counts, labels=segment_labels, autopct='%1.1f%%',
                                       colors=colors1, startangle=90)
    ax1.set_title('Account Portfolio Distribution', fontweight='bold')
    
    # Chart 2: Monthly Meetings by Segment
    meeting_counts = [account_data['monthly_meetings_by_segment'].get(seg, 0) for seg in segments]
    
    bars2 = ax2.bar(segment_labels, meeting_counts, color=colors1)
    ax2.set_title('Monthly Meetings by Account Segment', fontweight='bold')
    ax2.set_ylabel('Meetings per Month')
    ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, count in zip(bars2, meeting_counts):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{count:.1f}',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 3: Existing vs New Customer Time
    customer_types = ['Existing\nAccounts', 'New Customer\nOnboarding']
    existing_meetings = sum(account_data['monthly_meetings_by_segment'].values())
    onboarding_meetings = sum(account_data['onboarding_meetings'].values())
    customer_meetings = [existing_meetings, onboarding_meetings]
    
    colors3 = sns.color_palette("Set1", 2)
    bars3 = ax3.bar(customer_types, customer_meetings, color=colors3)
    ax3.set_title('Customer Meeting Distribution', fontweight='bold')
    ax3.set_ylabel('Meetings per Month')
    
    # Add value labels and percentages
    total_meetings = existing_meetings + onboarding_meetings
    for bar, count in zip(bars3, customer_meetings):
        height = bar.get_height()
        percentage = (count / total_meetings * 100) if total_meetings > 0 else 0
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{count:.1f}\n({percentage:.1f}%)',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 4: Account Load per SE
    distribution_data = ACCOUNT_RESULTS['distribution_analysis']
    
    load_metrics = ['Accounts\nper SE', 'Weekly Meetings\nper SE', 'Weekly Hours\nper SE']
    load_values = [
        distribution_data['accounts_per_se'],
        distribution_data['meetings_per_se_weekly'],
        distribution_data['hours_per_se_weekly']
    ]
    
    # Normalize values for comparison (different scales)
    normalized_values = []
    reference_values = [50, 10, 20]  # Reference points for scaling
    
    for value, ref in zip(load_values, reference_values):
        normalized_values.append((value / ref) * 100)
    
    colors4 = sns.color_palette("viridis", len(load_metrics))
    bars4 = ax4.bar(load_metrics, normalized_values, color=colors4)
    ax4.set_title('SE Account Load Analysis (% of Reference)', fontweight='bold')
    ax4.set_ylabel('Percentage of Reference Load')
    ax4.axhline(y=100, color='red', linestyle='--', alpha=0.7, label='Reference Level')
    ax4.legend()
    
    # Add actual value labels
    for bar, actual_value, ref_value in zip(bars4, load_values, reference_values):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{actual_value:.1f}\n(ref: {ref_value})',
                ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print account management summary
    print(f"\n📊 Account Management Summary:")
    print(f"   • Total Accounts: {account_data['total_accounts']}")
    print(f"   • Monthly Meetings: {account_data['total_monthly_meetings']:.1f}")
    print(f"   • Weekly Hours: {format_hours(account_data['weekly_account_hours'])}")
    print(f"   • Load per SE: {distribution_data['accounts_per_se']:.1f} accounts, {format_hours(distribution_data['hours_per_se_weekly'])}/week")
    
    return fig

def create_strategic_activities_chart():
    """Create visualizations for strategic activities allocation."""
    
    print("\n🎯 Creating Strategic Activities Chart")
    print("=" * 42)
    
    if 'STRATEGIC_RESULTS' not in globals():
        print("❌ Strategic results not available. Please run strategic activities calculations first.")
        return None
    
    strategic_data = STRATEGIC_RESULTS['strategic_summary']
    capacity_data = STRATEGIC_RESULTS['capacity_impact']
    
    setup_chart_style()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Strategic Activities Analysis', fontsize=16, fontweight='bold')
    
    # Chart 1: Strategic Activity Hours Breakdown
    activities = []
    activity_hours = []
    
    for activity, allocation in strategic_data['activity_allocation'].items():
        activities.append(activity.replace('_', ' ').title())
        activity_hours.append(allocation['total_hours'])
    
    colors1 = sns.color_palette("Set2", len(activities))
    bars1 = ax1.bar(activities, activity_hours, color=colors1)
    ax1.set_title('Strategic Activities Time Allocation', fontweight='bold')
    ax1.set_ylabel('Hours per Week')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels
    for bar, hours in zip(bars1, activity_hours):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(hours)}',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 2: IC vs Director Strategic Work Distribution
    roles = ['IC SEs', 'Directors']
    role_hours = [
        strategic_data['strategic_distribution']['total_ic_strategic_hours'],
        strategic_data['strategic_distribution']['total_director_strategic_hours']
    ]
    
    colors2 = sns.color_palette("Set1", 2)
    bars2 = ax2.bar(roles, role_hours, color=colors2)
    ax2.set_title('Strategic Work Distribution by Role', fontweight='bold')
    ax2.set_ylabel('Hours per Week')
    
    # Add value and percentage labels
    total_strategic = sum(role_hours)
    for bar, hours in zip(bars2, role_hours):
        height = bar.get_height()
        percentage = (hours / total_strategic * 100) if total_strategic > 0 else 0
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(hours)}\n({percentage:.1f}%)',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 3: Capacity Impact Analysis
    capacity_categories = ['IC\nCapacity', 'IC Strategic\nTime', 'IC Remaining\nCapacity', 
                          'Director\nCapacity', 'Director Strategic\nTime', 'Director IC\nCapacity']
    
    ic_total = capacity_data['total_capacities']['total_ic_capacity']
    ic_strategic = capacity_data['strategic_allocation']['ic_strategic_hours']
    ic_remaining = capacity_data['remaining_pipeline_capacity']['remaining_ic_capacity']
    
    director_total = capacity_data['total_capacities']['total_director_strategic_capacity']
    director_strategic = capacity_data['strategic_allocation']['director_strategic_hours']
    director_ic = capacity_data['total_capacities']['total_director_ic_capacity']
    
    capacity_values = [ic_total, ic_strategic, ic_remaining, director_total, director_strategic, director_ic]
    
    # Create grouped bar chart
    x_pos = np.arange(len(capacity_categories))
    colors3 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#1f77b4', '#ff7f0e', '#2ca02c']
    bars3 = ax3.bar(x_pos, capacity_values, color=colors3)
    
    ax3.set_title('Team Capacity Impact Analysis', fontweight='bold')
    ax3.set_ylabel('Hours per Week')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(capacity_categories, rotation=45)
    
    # Add value labels
    for bar, value in zip(bars3, capacity_values):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(value)}',
                ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Chart 4: Strategic Utilization Rates
    utilization_types = ['IC Strategic\nUtilization', 'Director Strategic\nUtilization', 'Overall Strategic\nUtilization']
    utilization_rates = [
        capacity_data['utilization_rates']['ic_strategic_utilization'],
        capacity_data['utilization_rates']['director_strategic_utilization'],
        capacity_data['utilization_rates']['overall_strategic_utilization']
    ]
    
    colors4 = sns.color_palette("viridis", len(utilization_types))
    bars4 = ax4.bar(utilization_types, utilization_rates, color=colors4)
    ax4.set_title('Strategic Utilization Rates', fontweight='bold')
    ax4.set_ylabel('Utilization Percentage')
    ax4.axhline(y=50, color='orange', linestyle='--', alpha=0.7, label='50% Utilization')
    ax4.axhline(y=75, color='red', linestyle='--', alpha=0.7, label='75% Utilization')
    ax4.legend()
    
    # Add value labels
    for bar, rate in zip(bars4, utilization_rates):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{rate:.1f}%',
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print strategic activities summary
    print(f"\n🎯 Strategic Activities Summary:")
    print(f"   • Total Strategic Hours: {format_hours(strategic_data['avg_strategic_hours'])}")
    print(f"   • IC Strategic Hours: {format_hours(strategic_data['strategic_distribution']['total_ic_strategic_hours'])}")
    print(f"   • Director Strategic Hours: {format_hours(strategic_data['strategic_distribution']['total_director_strategic_hours'])}")
    print(f"   • IC Utilization: {capacity_data['utilization_rates']['ic_strategic_utilization']:.1f}%")
    print(f"   • Director Utilization: {capacity_data['utilization_rates']['director_strategic_utilization']:.1f}%")
    
    return fig

def create_comprehensive_summary_chart():
    """Create a comprehensive summary visualization combining all workstreams."""
    
    print("\n📊 Creating Comprehensive Summary Chart")
    print("=" * 45)
    
    # Check for required data
    required_globals = ['PIPELINE_RESULTS', 'ACCOUNT_RESULTS', 'STRATEGIC_RESULTS']
    for global_var in required_globals:
        if global_var not in globals():
            print(f"❌ {global_var} not available. Please run all calculations first.")
            return None
    
    setup_chart_style()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('SE Team Workload Summary Dashboard', fontsize=16, fontweight='bold')
    
    # Gather all data
    pipeline_hours = PIPELINE_RESULTS['pipeline_summary']['total_pipeline_hours']
    account_hours = ACCOUNT_RESULTS['account_summary']['weekly_account_hours']
    strategic_hours = STRATEGIC_RESULTS['strategic_summary']['avg_strategic_hours']
    fixed_hours = sum(fixed_activity_params.values())
    
    total_ics = staffing_params['num_ic_ses']
    total_directors = staffing_params['num_directors']
    total_capacity = (total_ics + total_directors) * 40
    operational_capacity = total_ics * 40 + total_directors * 40 * (staffing_params['director_ic_percentage'] / 100)
    
    # Chart 1: Overall Workload Summary
    workstreams = ['Pipeline', 'Accounts', 'Strategic', 'Fixed\nActivities']
    workstream_hours = [pipeline_hours, account_hours, strategic_hours, fixed_hours]
    total_demand = sum(workstream_hours)
    
    colors1 = sns.color_palette("Set2", len(workstreams))
    bars1 = ax1.bar(workstreams, workstream_hours, color=colors1)
    ax1.set_title('Weekly Workload by Stream', fontweight='bold')
    ax1.set_ylabel('Hours per Week')
    
    # Add capacity line and utilization info
    ax1.axhline(y=operational_capacity, color='red', linestyle='--', alpha=0.7,
                label=f'Operational Capacity ({operational_capacity:.1f}h)')
    ax1.legend()
    
    # Add value labels
    for bar, hours in zip(bars1, workstream_hours):
        height = bar.get_height()
        percentage = (hours / operational_capacity * 100) if operational_capacity > 0 else 0
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{format_hours(hours)}\n({percentage:.1f}%)',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 2: Revenue Impact Analysis
    revenue_quarterly = PIPELINE_RESULTS['pipeline_summary']['new_logos_needed'] * revenue_params['average_selling_price']
    revenue_metrics = ['Quarterly\nRevenue Goal', 'Projected\nRevenue', 'Revenue per\nSE Hour']
    
    revenue_per_hour = revenue_quarterly / (13 * total_demand) if total_demand > 0 else 0
    revenue_values = [revenue_params['quarterly_revenue_goal'] / 1000000,  # Convert to millions
                     revenue_quarterly / 1000000,
                     revenue_per_hour / 1000]  # Convert to thousands
    
    colors2 = sns.color_palette("viridis", len(revenue_metrics))
    bars2 = ax2.bar(revenue_metrics, revenue_values, color=colors2)
    ax2.set_title('Revenue Impact Analysis', fontweight='bold')
    ax2.set_ylabel('Revenue ($ Millions / $ Thousands)')
    
    # Add value labels
    labels = [f'${revenue_params["quarterly_revenue_goal"]/1000000:.1f}M',
             f'${revenue_quarterly/1000000:.1f}M',
             f'${revenue_per_hour/1000:.1f}K/hr']
    
    for bar, label in zip(bars2, labels):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                label,
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 3: Team Efficiency Metrics
    efficiency_metrics = ['Team\nUtilization', 'Pipeline\nEfficiency', 'Account\nLoad Factor']
    
    team_utilization = (total_demand / operational_capacity * 100) if operational_capacity > 0 else 0
    pipeline_efficiency = (pipeline_hours / total_demand * 100) if total_demand > 0 else 0
    account_load_factor = (ACCOUNT_RESULTS['distribution_analysis']['accounts_per_se'] / 50 * 100)  # Reference: 50 accounts
    
    efficiency_values = [team_utilization, pipeline_efficiency, account_load_factor]
    
    colors3 = sns.color_palette("Set1", len(efficiency_metrics))
    bars3 = ax3.bar(efficiency_metrics, efficiency_values, color=colors3)
    ax3.set_title('Team Efficiency Metrics', fontweight='bold')
    ax3.set_ylabel('Percentage')
    
    # Add reference lines
    ax3.axhline(y=100, color='orange', linestyle='--', alpha=0.7, label='100% Reference')
    ax3.axhline(y=75, color='green', linestyle='--', alpha=0.7, label='75% Target')
    ax3.legend()
    
    # Add value labels
    for bar, value in zip(bars3, efficiency_values):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{value:.1f}%',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 4: Scenario Analysis
    scenarios = ['Current\nState', 'High\nGrowth', 'Resource\nConstrained']
    
    # Current state
    current_total = total_demand
    
    # High growth scenario (25% more pipeline)
    high_growth_total = total_demand + (pipeline_hours * 0.25)
    
    # Resource constrained (1 fewer IC)
    constrained_capacity = operational_capacity - 40
    constrained_utilization = (total_demand / constrained_capacity * 100) if constrained_capacity > 0 else 100
    
    scenario_values = [
        team_utilization,
        (high_growth_total / operational_capacity * 100) if operational_capacity > 0 else 100,
        constrained_utilization
    ]
    
    colors4 = sns.color_palette("plasma", len(scenarios))
    bars4 = ax4.bar(scenarios, scenario_values, color=colors4)
    ax4.set_title('Scenario Analysis - Team Utilization', fontweight='bold')
    ax4.set_ylabel('Utilization Percentage')
    
    # Add capacity warning line
    ax4.axhline(y=100, color='red', linestyle='--', alpha=0.7, label='100% Capacity')
    ax4.legend()
    
    # Add value labels
    for bar, value in zip(bars4, scenario_values):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{value:.1f}%',
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print comprehensive summary
    print(f"\n📈 Comprehensive Workload Summary:")
    print(f"   • Total Weekly Demand: {format_hours(total_demand)}")
    print(f"   • Operational Capacity: {format_hours(operational_capacity)}")
    print(f"   • Team Utilization: {team_utilization:.1f}%")
    print(f"   • Available Capacity: {format_hours(max(0, operational_capacity - total_demand))}")
    print(f"   • Revenue Target: {format_currency(revenue_params['quarterly_revenue_goal'])}/quarter")
    print(f"   • Revenue per Hour: {format_currency(revenue_per_hour)}/hour")
    
    return fig

# ===================================
# MAIN VISUALIZATION EXECUTION
# ===================================

def run_all_core_visualizations():
    """Execute all core visualization functions in sequence."""
    
    print("🎨 Running All Core Visualizations")
    print("=" * 50)
    
    # Check if all required data is available
    required_globals = ['PIPELINE_RESULTS', 'ACCOUNT_RESULTS', 'STRATEGIC_RESULTS']
    missing_data = [var for var in required_globals if var not in globals()]
    
    if missing_data:
        print(f"❌ Missing required data: {', '.join(missing_data)}")
        print("📝 Please run all business logic calculations first (Steps 3-5)")
        return False
    
    visualization_results = {}
    
    try:
        # Create each visualization
        print("\n1. Creating Capacity Utilization Chart...")
        fig1 = create_capacity_utilization_chart()
        visualization_results['capacity_chart'] = fig1
        
        print("\n2. Creating Pipeline Funnel Chart...")
        fig2 = create_pipeline_funnel_chart()
        visualization_results['pipeline_chart'] = fig2
        
        print("\n3. Creating Account Management Chart...")
        fig3 = create_account_management_chart()
        visualization_results['account_chart'] = fig3
        
        print("\n4. Creating Strategic Activities Chart...")
        fig4 = create_strategic_activities_chart()
        visualization_results['strategic_chart'] = fig4
        
        print("\n5. Creating Comprehensive Summary Chart...")
        fig5 = create_comprehensive_summary_chart()
        visualization_results['summary_chart'] = fig5
        
        print("\n✅ All core visualizations created successfully!")
        print("📊 5 comprehensive charts generated")
        
        return visualization_results
        
    except Exception as e:
        print(f"\n❌ Error creating visualizations: {str(e)}")
        return False

# Check if we have all the required data and run visualizations
if all(var in globals() for var in ['PIPELINE_RESULTS', 'ACCOUNT_RESULTS', 'STRATEGIC_RESULTS']):
    # Run all visualizations
    VISUALIZATION_RESULTS = run_all_core_visualizations()
    
    if VISUALIZATION_RESULTS:
        print("\n🎉 Core Visualizations (Step 7) completed successfully!")
        print("📝 Ready for Step 8: Advanced Dashboard")
    else:
        print("\n❌ Some visualizations failed to generate")
else:
    print("📊 Core visualizations framework ready")
    print("⚠️  Business logic calculations must be completed first")
    print("📝 Run cells 8-14 to generate required data, then re-run this cell")

---

# 9. Executive Dashboard {#executive-dashboard}

*Professional dashboard with comprehensive business insights*

In [ ]:
# Placeholder for executive dashboard
# Will be implemented in Step 11

print("📈 Executive dashboard section ready for implementation")

---

# 10. Results Summary & Export {#results-summary}

*Executive summary generation and export capabilities*

In [ ]:
# Placeholder for results summary and export
# Will be implemented in Step 12

print("📋 Results summary & export section ready for implementation")

---

# 11. Testing Framework {#testing}

*Comprehensive validation and testing of all calculations*

In [ ]:
# Placeholder for testing framework
# Will be implemented throughout all steps

print("🧪 Testing framework section ready for implementation")

---

# 12. Usage Examples {#examples}

*Sample scenarios and practical use cases*

In [ ]:
# Placeholder for usage examples
# Will be implemented in final step

print("💡 Usage examples section ready for implementation")

---

## Import Verification Test

*Basic test to verify all imports are working correctly*

In [ ]:
# Test all imported libraries
def test_imports():
    """Verify all required libraries are properly imported and functional."""
    test_results = []
    
    # Test pandas
    try:
        test_df = pd.DataFrame({'test': [1, 2, 3]})
        assert len(test_df) == 3
        test_results.append("✅ pandas: Working correctly")
    except Exception as e:
        test_results.append(f"❌ pandas: Error - {str(e)}")
    
    # Test numpy
    try:
        test_array = np.array([1, 2, 3])
        assert np.sum(test_array) == 6
        test_results.append("✅ numpy: Working correctly")
    except Exception as e:
        test_results.append(f"❌ numpy: Error - {str(e)}")
    
    # Test matplotlib
    try:
        fig, ax = plt.subplots(1, 1, figsize=(4, 3))
        ax.plot([1, 2, 3], [1, 4, 2])
        plt.close(fig)
        test_results.append("✅ matplotlib: Working correctly")
    except Exception as e:
        test_results.append(f"❌ matplotlib: Error - {str(e)}")
    
    # Test seaborn
    try:
        current_style = sns.axes_style()
        assert isinstance(current_style, dict)
        test_results.append("✅ seaborn: Working correctly")
    except Exception as e:
        test_results.append(f"❌ seaborn: Error - {str(e)}")
    
    # Test ipywidgets
    try:
        test_widget = widgets.IntSlider(value=5, min=0, max=10)
        assert test_widget.value == 5
        test_results.append("✅ ipywidgets: Working correctly")
    except Exception as e:
        test_results.append(f"❌ ipywidgets: Error - {str(e)}")
    
    # Display results
    print("\n🔍 Import Verification Results:")
    print("=" * 40)
    for result in test_results:
        print(result)
    
    # Summary
    success_count = sum(1 for result in test_results if result.startswith("✅"))
    total_count = len(test_results)
    
    print("\n📊 Summary:")
    print(f"   • {success_count}/{total_count} libraries working correctly")
    
    if success_count == total_count:
        print("\n🎉 All imports successful! Ready to proceed with implementation.")
        return True
    else:
        print("\n⚠️  Some imports failed. Please check your environment.")
        return False

# Run the test
test_imports()

---

## Next Steps

✅ **Step 1 Complete**: Basic notebook structure created with all required sections

### Ready for Implementation:
- **Step 2**: Helper Functions Framework - Core calculation functions
- **Step 3**: Parameter Management - Input parameter definitions and validation

### Implementation Notes:
- All libraries imported and tested successfully
- Professional styling and configuration applied
- Clear section structure with table of contents navigation
- Placeholder cells ready for modular development
- Testing framework initialized

**📝 Continue with Step 2 to implement the helper functions framework.**